# 🧱 Assignment 1 — Assembling Lego Kits

## Authors: Francisco Pinto, João Soares, João Viterbo

**FEUP — Computer Vision 2025/26**  
This notebook performs all tasks required in the assignment, using OpenCV and Python in Google Colab.

All results are displayed inline and saved under `/results/`.

## Data Load and Directory Creation

In [ ]:
# --- SYSTEM SETUP AND DATA DOWNLOAD (Drive-mounted version) ---

import os, shutil
from google.colab import drive

# Create folders
os.makedirs("data", exist_ok=True)
os.makedirs("results", exist_ok=True)

# 1️⃣ Mount your Google Drive
drive.mount('/content/drive')

# 2️⃣ Define source and destination
# Replace the path below with the exact path where your "Calibration", "Fault", "Isolated", "Kit" folders live.
# Tip: In Colab's left panel, navigate to "drive/MyDrive", right-click the folder → "Copy path"
src = '/content/drive/MyDrive/LEGO_DATA'   # 🔁 adjust this
dst = '/content/data'

# 3️⃣ Copy recursively (keeps subfolders)
os.makedirs(dst, exist_ok=True)
shutil.copytree(src, dst, dirs_exist_ok=True)

print("✅ All project data copied recursively to /content/data")

# 4️⃣ (Optional) Verify structure
for root, dirs, files in os.walk(dst):
    for f in files:
        print(os.path.join(root, f))


In [ ]:
# --- SYSTEM SETUP AND DATA DOWNLOAD (copy link version - not working) ---
'''
import os, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt, glob
from google.colab import drive
import gdown
from IPython.display import display, Image

# Create folders
os.makedirs('data', exist_ok=True)
os.makedirs('results', exist_ok=True)

# Public Drive folder ID
folder_url = 'https://drive.google.com/drive/folders/1OCikW87wfB8TGvcw4qtL0bdCelWt3-wV?usp=sharing'
!gdown --folder $folder_url -O ./data --remaining-ok

print('✅ Data downloaded successfully.')
'''

## 1️⃣ Camera Calibration

We estimate intrinsic and extrinsic parameters for the camera using the provided chessboard images.

In [ ]:
# auxiliary code to find what is the best pattern size
import cv2
import glob
import matplotlib.pyplot as plt

# Pick one sample image from the first folder
test_img_path = glob.glob('./data/Calibration/calib1/*.png')[0]
print("Testing image:", test_img_path)

img = cv2.imread(test_img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

found_any = False
for cols in range(5, 13):  # test 5–12 columns
    for rows in range(4, 10):  # test 4–9 rows
        ret, corners = cv2.findChessboardCorners(gray, (cols, rows), None)
        if ret:
            found_any = True
            print(f"✅ Pattern found with size ({cols}, {rows})")
            vis = img.copy()
            cv2.drawChessboardCorners(vis, (cols, rows), corners, ret)
            plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
            plt.title(f"Detected pattern size: ({cols}, {rows})")
            plt.axis('off')
            plt.show()
            break
    if found_any:
        break

if not found_any:
    print("❌ No chessboard pattern detected in this image — might not be a standard calibration grid.")


In [ ]:
# auxiliary code to verify the pattern size

import glob, cv2

pattern_size = (11, 8)  # or (6, 7) depending on what you decided
valid = 0
images = glob.glob('./data/Calibration/calib2/*.png')
for img_path in images:
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, _ = cv2.findChessboardCorners(gray, pattern_size, None)
    if ret:
        valid += 1
print(f"✅ {valid}/{len(images)} images with detected corners in calib2")



In [ ]:
# Intrinsic values calculation with a resume of results.

import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt

def calibrate(folder):
    # Chessboard configuration (12x9 squares → 11x8 inner corners)
    pattern_size = (11, 8)
    square_size = 15.0  # millimeters

    # Prepare 3D object points with physical scale
    objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    objp *= square_size

    objpoints, imgpoints = [], []
    images = glob.glob(f"{folder}/*.png")
    img_size = None

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
        if ret:
            corners2 = cv2.cornerSubPix(
                gray, corners, (11, 11), (-1, -1),
                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            )
            objpoints.append(objp)
            imgpoints.append(corners2)
            img_size = gray.shape[::-1]
        else:
            print(f"⚠️ Chessboard not detected in {fname}")

    if len(objpoints) == 0 or img_size is None:
        raise ValueError(f"No valid calibration images found in {folder}")

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

    # Compute reprojection error
    total_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        total_error += error
    mean_error = total_error / len(objpoints)

    return mtx, dist, mean_error

# === Run for all three folders ===
for folder in ['calib1', 'calib2', 'calib3']:
    fpath = f'./data/Calibration/{folder}'
    try:
        mtx, dist, err = calibrate(fpath)
        print(f"\n📁 {folder}")
        print("Intrinsic matrix (K):\n", mtx)
        print("Distortion coefficients:\n", dist.ravel())
        print(f"Mean reprojection error: {err:.4f}")
    except ValueError as e:
        print(f"⚠️ Skipping {folder}: {e}")


## Results - Answer to Question 1a)

The calibration from calib2 was selected because it achieved the lowest mean reprojection error (0.0074 px), indicating the most accurate mapping between 3-D object points and image points.
The intrinsics are also consistent with the other attempts, confirming that the solution is stable and physically plausible.

In [ ]:
# Intrinsic values calculation with the print of the images.

import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt

def calibrate(folder):
    # correct pattern size found earlier
    pattern_size = (11, 8)  # (columns, rows) of inner corners

    # prepare object points
    objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

    objpoints = []  # 3D points
    imgpoints = []  # 2D points

    images = glob.glob(f"{folder}/*.png")
    img_size = None

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(
                gray, corners, (11, 11), (-1, -1),
                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            )
            imgpoints.append(corners2)
            img_size = gray.shape[::-1]

            img_vis = cv2.drawChessboardCorners(img.copy(), pattern_size, corners2, ret)
            plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()
        else:
            print(f"⚠️ Chessboard not detected in {fname}")

    if len(objpoints) == 0 or img_size is None:
        raise ValueError(f"No valid calibration images found in {folder}")

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, img_size, None, None
    )

    total_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        total_error += error
    mean_error = total_error / len(objpoints)

    print(f"Used {len(objpoints)} valid images for calibration in {folder}")

    return mtx, dist, mean_error

for folder in ['calib1', 'calib2', 'calib3']:
    fpath = f'./data/Calibration/{folder}'
    try:
        mtx, dist, err = calibrate(fpath)
        print(f'✅ {folder}: reprojection error = {err:.4f}')
    except ValueError as e:
        print(f'⚠️ Skipping {folder} → {e}')


### Extrinsic calibration (final setup)

In [ ]:
# This code does more than requested. It is using all images to do the calibration and not only the
# final_setup.png image
# EXTRA WORK DONE

import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt

# === PARAMETERS ===
pattern_size = (11, 8)    # inner corners per chessboard row/col
square_size = 15.0        # mm per square
folder = './data/Calibration/calib2'
axis_len = 30             # length of drawn 3D axes (mm)

# === LOAD CAMERA INTRINSICS ===
# You must have defined or imported a calibrate(folder) function
K, dist, _ = calibrate(folder)

# === PREPARE CHESSBOARD POINTS (world coordinates) ===
objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
objp *= square_size

# === PROCESS ALL IMAGES ===
images = glob.glob(f"{folder}/*.png")
extrinsics = []
reproj_errors = []

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
    if not ret:
        print(f"⚠️ Corners not detected in {fname}")
        continue

    # Refine corner locations
    corners2 = cv2.cornerSubPix(
        gray, corners, (11, 11), (-1, -1),
        (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    )

    # Compute extrinsics
    success, rvec, tvec = cv2.solvePnP(objp, corners2, K, dist)
    if not success:
        print(f"⚠️ PnP failed for {fname}")
        continue

    # Compute reprojection error
    projected, _ = cv2.projectPoints(objp, rvec, tvec, K, dist)
    error = cv2.norm(corners2, projected, cv2.NORM_L2) / len(projected)
    reproj_errors.append(error)
    extrinsics.append((fname, rvec, tvec, error))

    # === VISUALIZE AXES ===
    axis = np.float32([[axis_len, 0, 0],
                       [0, axis_len, 0],
                       [0, 0, -axis_len]]).reshape(-1, 3)
    imgpts, _ = cv2.projectPoints(axis, rvec, tvec, K, dist)
    corner = tuple(corners2[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255, 0, 0), 3)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0, 255, 0), 3)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0, 0, 255), 3)
    plt.figure(figsize=(6, 4))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"{fname.split('/')[-1]} | reproj err: {error:.3f}")
    plt.axis('off')
    plt.show()

print(f"\n✅ Extrinsics computed for {len(extrinsics)}/{len(images)} images.")

# === SELECT BEST IMAGE (lowest reprojection error) ===
if not extrinsics:
    raise ValueError("No valid extrinsics computed. Check chessboard detections.")

best_fname, best_rvec, best_tvec, best_err = min(extrinsics, key=lambda x: x[3])
print(f"\n🏆 Best image: {best_fname.split('/')[-1]} (reproj error={best_err:.4f})")

# === COMPUTE FINAL EXTRINSIC MATRIX ===
R, _ = cv2.Rodrigues(best_rvec)
extrinsic_matrix = np.hstack([R, best_tvec])
print("\nRotation vector (rvec):\n", best_rvec)
print("Translation vector t (mm):\n", best_tvec)
print("Extrinsic matrix [R|t]:\n", extrinsic_matrix)

# === PIXEL-TO-MM CONVERSION ===
img_best = cv2.imread(best_fname)
gray_best = cv2.cvtColor(img_best, cv2.COLOR_BGR2GRAY)
ret, corners_best = cv2.findChessboardCorners(gray_best, pattern_size, None)
corners_best = cv2.cornerSubPix(
    gray_best, corners_best, (11, 11), (-1, -1),
    (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
)

# Local pixel spacing
cols, rows = pattern_size
dists = []
for r in range(rows):
    for c in range(cols - 1):
        i = r * cols + c
        dists.append(np.linalg.norm(corners_best[i + 1] - corners_best[i]))
for r in range(rows - 1):
    for c in range(cols):
        i = r * cols + c
        j = (r + 1) * cols + c
        dists.append(np.linalg.norm(corners_best[j] - corners_best[i]))

px_per_square = float(np.mean(dists))
mm_per_px = square_size / px_per_square
print(f"\n📏 Pixel-to-mm conversion ≈ {mm_per_px:.6f} mm/px (from avg square spacing)")


In [ ]:
# This code does what is requested
# uses final_setup.png to calculate the variables

import cv2, numpy as np, matplotlib.pyplot as plt

# 1) Load best intrinsics (from calib3)
K, dist, _ = calibrate('./data/Calibration/calib2')

# 2) Prepare object points in mm for 11x8 inner corners
pattern_size = (11, 8)
square_size = 15.0  # mm
objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
objp *= square_size

# 3) Load final setup image and detect corners
img = cv2.imread('./data/Calibration/final_setup.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
if not ret:
    raise ValueError("Chessboard not detected in final setup image.")
corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1),
                            (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3))

# 4) Solve for extrinsics
ok, rvec, tvec = cv2.solvePnP(objp, corners2, K, dist)
R, _ = cv2.Rodrigues(rvec)
extrinsic_matrix = np.hstack([R, tvec])   # 3x4 [R|t]

print("Rotation vector (rvec):\n", rvec)
print("Translation vector t (mm):\n", tvec)
print("Extrinsic matrix [R|t]:\n", extrinsic_matrix)

# 5) Pixel-to-millimeter conversion (local estimate from chessboard spacing)
# Use both horizontal and vertical neighbor distances for robustness
dists = []
cols, rows = pattern_size
for r in range(rows):
    for c in range(cols-1):  # horizontal neighbors
        i = r*cols + c
        dists.append(np.linalg.norm(corners2[i+1]-corners2[i]))
for r in range(rows-1):
    for c in range(cols):    # vertical neighbors
        i = r*cols + c
        j = (r+1)*cols + c
        dists.append(np.linalg.norm(corners2[j]-corners2[i]))

px_per_square = float(np.mean(dists))
mm_per_px = square_size / px_per_square
print(f"Pixel-to-mm conversion ≈ {mm_per_px:.6f} mm/px  (from avg square spacing)")


## Results - Answer to Question 1b)

Extrinsic matrix

[R|t]:

 [[-9.98391200e-01 -1.22530188e-02 -5.53613199e-02  6.41626142e+01]

 [ 8.67550580e-03 -9.97886105e-01  6.44054112e-02  3.56702439e+01]

 [-5.60334526e-02  6.38215083e-02  9.96387007e-01  2.66735075e+02]]

 Pixel-to-mm conversion ≈ 0.330046 mm/px  (from avg square spacing)

## 2️⃣ Isolated Bricks Analysis

In [ ]:
# CLEAN RESULTS
# USE TO CLEAN ALL RESULTS FOLDER

import os, shutil

def clear_output_folder(folder_path):
    """Delete all files and subfolders inside the results directory."""
    if os.path.exists(folder_path):
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)
            try:
                if os.path.isfile(item_path) or os.path.islink(item_path):
                    os.unlink(item_path)
                elif os.path.isdir(item_path):
                    shutil.rmtree(item_path)
            except Exception as e:
                print(f"⚠️ Error deleting {item_path}: {e}")
    else:
        os.makedirs(folder_path)

# --- Clear output folders before starting ---
clear_output_folder("results")
print("🧹 Old results deleted. Ready to run fresh detection.")


In [ ]:
# SMALL TEST TO DEFINE ROI MANUALLY

img = cv2.imread('./data/Isolated/colored_bricks.png')

# Crop region (y1:y2, x1:x2)
roi = img[0:400, 0:300]

# Show ROI to confirm
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
plt.title('Selected ROI')
plt.axis('off')
plt.show()


In [ ]:
# DETECT REGION OF INTEREST COMPRISING ALL THE BRICKS

import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_brick_roi(img_bgr, show_steps=True):
    """Automatically detect the ROI that contains all colored bricks."""

    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    # Define HSV color ranges for the main LEGO colors
    color_ranges = {
        'red1':   [(0, 120, 70), (10, 255, 255)],
        'red2':   [(170, 120, 70), (180, 255, 255)],
        'blue':   [(90, 80, 50), (130, 255, 255)],
        'green':  [(40, 60, 50), (85, 255, 255)],
        'yellow': [(20, 100, 100), (35, 255, 255)],
        'white':  [(0, 0, 200), (180, 60, 255)]  # low saturation, high brightness
    }

    # Combine all color masks into one
    combined_mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
    for name, (lo, hi) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lo), np.array(hi))
        combined_mask = cv2.bitwise_or(combined_mask, mask)

    # Optional cleaning (remove small noise, fill gaps)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, k)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, k)

    # Find all contours in the combined mask
    cnts, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not cnts:
        print("⚠️ No colored regions found — check color ranges.")
        return None, None, combined_mask

    # Compute bounding box that encloses all color contours
    all_points = np.vstack(cnts)  # stack all contour points together
    x, y, w, h = cv2.boundingRect(all_points)
    roi = img_bgr[y:y+h, x:x+w]

    if show_steps:
        # Display mask and ROI overlay
        vis = img_bgr.copy()
        cv2.rectangle(vis, (x, y), (x+w, y+h), (0,255,0), 3)

        plt.figure(figsize=(12,5))
        plt.subplot(1,2,1)
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title("Detected ROI on Original")
        plt.axis('off')

        plt.subplot(1,2,2)
        plt.imshow(combined_mask, cmap='gray')
        plt.title("Combined Color Mask")
        plt.axis('off')
        plt.show()

    return roi, (x, y, w, h), combined_mask


# --- Example usage ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
roi, bbox, mask = detect_brick_roi(img)

if roi is not None:
    print(f"Detected ROI bounding box: x={bbox[0]}, y={bbox[1]}, w={bbox[2]}, h={bbox[3]}")
    plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
    plt.title("Cropped ROI (auto detected)")
    plt.axis('off')
    plt.show()


In [ ]:
# DETECT ROI BY COLOR OF BRICKS

import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_color_rois(img_bgr, show=True):
    """Detect one ROI per color (red, blue, green, yellow, white)."""

    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    color_ranges = {
        'red1':   [(0, 120, 70), (10, 255, 255)],
        'red2':   [(170, 120, 70), (180, 255, 255)],
        'blue':   [(90, 80, 50), (130, 255, 255)],
        'green':  [(40, 60, 50), (85, 255, 255)],
        'yellow': [(20, 100, 100), (35, 255, 255)],
        #'white':  [(0, 0, 200), (180, 60, 255)]
        'white':  [(0, 0, 200), (30, 50, 255)]  # my settings for white
    }

    color_bgr_map = {
        'red': (0, 0, 255),
        'blue': (255, 0, 0),
        'green': (0, 255, 0),
        'yellow': (0, 255, 255),
        'white': (180, 180, 180)
    }

    color_rois = {}
    vis = img_bgr.copy()

    for name, (lo, hi) in color_ranges.items():
        base = 'red' if 'red' in name else name
        mask = cv2.inRange(hsv, np.array(lo), np.array(hi))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not cnts:
            continue
        all_points = np.vstack(cnts)
        x, y, w, h = cv2.boundingRect(all_points)
        color_rois[base] = (x, y, w, h)
        cv2.rectangle(vis, (x, y), (x+w, y+h), color_bgr_map[base], 2)
        cv2.putText(vis, base, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr_map[base], 1)

    if show:
        plt.figure(figsize=(10, 8))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title("Per-color ROIs")
        plt.axis('off')
        plt.show()

    return color_rois


# --- Example usage ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
color_rois = detect_color_rois(img)
print(color_rois)

# Optional: crop and show one ROI
for color, (x, y, w, h) in color_rois.items():
    roi = img[y:y+h, x:x+w]
    plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
    plt.title(f"{color} ROI")
    plt.axis('off')
    plt.show()


In [ ]:
# EXPERIMENT FOCUSING ONLY ON THE WHITE BRICKS
# EXPERIMENT WITH MASK

import cv2
import numpy as np
import matplotlib.pyplot as plt

# --- Step 1: Load image ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
if img is None:
    raise FileNotFoundError("Image not found at ./data/Isolated/colored_bricks.png")

# --- Step 2: Convert to HSV ---
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# --- Step 3: Define white color range ---
# White = low saturation (S) and high brightness (V)
lower_white = np.array([0, 0, 200])
upper_white = np.array([180, 60, 255])

# --- Step 4: Create binary mask ---
mask_white = cv2.inRange(hsv, lower_white, upper_white)

# --- Step 5: Morphological cleanup ---
# This removes small dots and fills tiny holes
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)
mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel)

# --- Step 6: Optional blur to smooth edges ---
mask_white = cv2.GaussianBlur(mask_white, (3, 3), 0)

# --- Step 7: Display results ---
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(mask_white, cmap='gray')
plt.title("White Mask")
plt.axis('off')
plt.show()

# --- Step 8: Print basic mask stats ---
num_white_pixels = cv2.countNonZero(mask_white)
mask_area_ratio = num_white_pixels / (mask_white.shape[0] * mask_white.shape[1])
print(f"Number of white pixels detected: {num_white_pixels}")
print(f"White area ratio: {mask_area_ratio:.4f}")

# Optional: save mask for inspection
cv2.imwrite('results/mask_white.png', mask_white)
print("Saved mask to results/mask_white.png")


In [ ]:
# EXPERIMENT FOCUSING ONLY ON THE WHITE BRICKS
# TRY TO DETECT NUMBER OF BRICKS - UNSUCCEED !!!!

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Step 1: Load image ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
if img is None:
    raise FileNotFoundError("Image not found at ./data/Isolated/colored_bricks.png")

# --- Step 2: Convert to HSV ---
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# --- Step 3: Define white color range ---
# White = low saturation, high brightness
lower_white = np.array([0, 0, 200])
upper_white = np.array([180, 60, 255])

# --- Step 4: Create binary mask ---
mask_white = cv2.inRange(hsv, lower_white, upper_white)

# --- Step 5: Morphological cleanup ---
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)
mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel)

# --- Step 6: Find contours (white regions) ---
contours, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# --- Step 7: Draw bounding boxes for detected white bricks ---
vis = img.copy()
min_area = 500  # pixels; ignore tiny noise blobs
white_bricks = 0

for c in contours:
    area = cv2.contourArea(c)
    if area < min_area:
        continue
    x, y, w, h = cv2.boundingRect(c)
    white_bricks += 1
    cv2.rectangle(vis, (x, y), (x + w, y + h), (180, 180, 180), 2)
    cv2.putText(vis, f'white #{white_bricks}', (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 1)

# --- Step 8: Display results ---
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.imshow(mask_white, cmap='gray')
plt.title('White Mask')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.title(f'Detected White Bricks ({white_bricks} found)')
plt.axis('off')
plt.show()

# --- Step 9: Print stats ---
num_white_pixels = cv2.countNonZero(mask_white)
mask_area_ratio = num_white_pixels / (mask_white.shape[0] * mask_white.shape[1])
print(f"Detected white bricks: {white_bricks}")
print(f"White pixels: {num_white_pixels} ({mask_area_ratio:.4%} of image)")

# --- Step 10: Save outputs ---
os.makedirs('results', exist_ok=True)
cv2.imwrite('results/mask_white.png', mask_white)
cv2.imwrite('results/white_bricks_detected.png', vis)
print("Saved mask and annotated image in 'results/' folder.")


In [ ]:
# EXPERIMENT FOCUSING ONLY ON THE WHITE BRICKS
# TRY USING EDGES
# TRY TO DETECT NUMBER OF BRICKS - UNSUCCEED !!!!

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Step 1: Load image ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
if img is None:
    raise FileNotFoundError("Image not found")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# --- Step 2: Enhance edges with local contrast ---
# Adaptive threshold detects slight brightness changes (edges/shadows)
th = cv2.adaptiveThreshold(
    gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,
    51, 4  # block size, constant subtracted; tune C between 2–6
)

# --- Step 3: Clean the mask ---
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
mask = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# --- Step 4: Find contours of potential bricks ---
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
vis = img.copy()

min_area = 1000  # remove tiny noise
white_bricks = 0

for c in contours:
    area = cv2.contourArea(c)
    if area < min_area:
        continue

    x, y, w, h = cv2.boundingRect(c)
    aspect = max(w, h) / (min(w, h) + 1e-6)
    fill_ratio = area / (w*h + 1e-6)

    # Filter by rectangularity and reasonable aspect
    if 0.6 < fill_ratio < 1.1 and 0.5 < aspect < 3.0:
        white_bricks += 1
        cv2.rectangle(vis, (x, y), (x+w, y+h), (180, 180, 180), 2)
        cv2.putText(vis, f'white #{white_bricks}', (x, y-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 1)

# --- Step 5: Display results ---
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.imshow(mask, cmap='gray')
plt.title('Adaptive Mask (edges of white bricks)')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.title(f'Detected White Bricks ({white_bricks} found)')
plt.axis('off')
plt.show()

# --- Step 6: Save and report ---
os.makedirs('results', exist_ok=True)
cv2.imwrite('results/mask_white_adaptive.png', mask)
cv2.imwrite('results/white_bricks_detected_adaptive.png', vis)

print(f"Detected white bricks: {white_bricks}")
print("Results saved in 'results/' folder.")


In [ ]:
# EXPERIMENT FOCUSING ONLY ON THE WHITE BRICKS
# FIRST APPLY MASK
# SECOND TRY TO DETECT EDGES
# TRY TO DETECT NUMBER OF BRICKS - UNSUCCEED !!!!

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Step 1: Load mask image ---
mask = cv2.imread('results/mask_white.png', cv2.IMREAD_GRAYSCALE)
if mask is None:
    raise FileNotFoundError("mask_white.png not found in results/")

# --- Step 2: Edge detection ---
# Canny detects transitions in the mask (useful if mask still includes background)
edges = cv2.Canny(mask, 50, 150)

# --- Step 3: Clean edges ---
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

# --- Step 4: Find contours of connected white regions ---
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# --- Step 5: Analyze and visualize ---
detected = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
min_area = 800  # ignore small fragments
white_bricks = 0

for c in contours:
    area = cv2.contourArea(c)
    if area < min_area:
        continue
    x, y, w, h = cv2.boundingRect(c)
    aspect = max(w, h)/(min(w, h)+1e-6)
    fill_ratio = area/(w*h+1e-6)
    # simple shape filter for brick-like blobs
    if 0.5 < aspect < 3.5 and fill_ratio > 0.55:
        white_bricks += 1
        cv2.rectangle(detected, (x, y), (x+w, y+h), (200,200,200), 2)
        cv2.putText(detected, f'white #{white_bricks}', (x, y-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

# --- Step 6: Show results ---
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.imshow(mask, cmap='gray'); plt.title('Input mask_white.png'); plt.axis('off')
plt.subplot(1,3,2)
plt.imshow(edges, cmap='gray'); plt.title('Edges'); plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(cv2.cvtColor(detected, cv2.COLOR_BGR2RGB))
plt.title(f'Detected White Bricks ({white_bricks})')
plt.axis('off')
plt.show()

# --- Step 7: Save outputs ---
os.makedirs('results', exist_ok=True)
cv2.imwrite('results/mask_white_edges.png', edges)
cv2.imwrite('results/white_bricks_from_mask.png', detected)
print(f"Detected {white_bricks} white bricks. Results saved in 'results/'.")


## FAILING TO DETECT WHITE PIECES

Since the previous experiences where not going well a different approach was taken.

There was no problem in detecting the colored ones, but for the whites it was not working.

When the mask was applied I the human could identify the brick even with the "noise" caused by the background. And a dintinctive feature was prevalent, the stud.

So the idea was, lets attack the image by pieces. Lets use the studs to create ROI (smaller ones) that allow then to apply the detection of the pieces.

The studs could be detected almost 100%. It was not perfect but were enough to apply a cluterization to create smaller regions (ROI).

It was oberved that some cluster have an overlap, so a treatment was applied to eliminate overlaps.

Then the MASK was applied by ROI by color and it worked 100%.


In [ ]:
# DETECT THE PIECES BY THE STUDS
# THIS WAS DONE SINCE USING ONLY MASK COLOR OVER ALL THE IMAGE WAS NOT WORKING

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Step 1: Load and prepare image ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
if img is None:
    raise FileNotFoundError("Image not found")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

# --- Step 2: Detect circles using HoughCircles ---
# Adjust these parameters depending on stud size and contrast
circles = cv2.HoughCircles(
    gray,
    cv2.HOUGH_GRADIENT,
    dp=1.2,             # inverse accumulator ratio (1.0–2.0 typical)
    minDist=25,         # minimum distance between circle centers (adjust!)
    param1=100,         # Canny high threshold
    param2=15,          # smaller = more sensitive, but more false positives - initial 20
    minRadius=6,        # min stud radius in pixels
    maxRadius=15        # max stud radius
)

# --- Step 3: Draw detected circles ---
vis = img.copy()
stud_centers = []

if circles is not None:
    circles = np.uint16(np.around(circles))
    for (x, y, r) in circles[0, :]:
        cv2.circle(vis, (x, y), r, (0, 255, 0), 2)
        cv2.circle(vis, (x, y), 2, (0, 0, 255), 3)
        stud_centers.append((x, y, r))

print(f"Detected {len(stud_centers)} studs (circular bumps).")

# --- Step 4: Display results ---
plt.figure(figsize=(10,8))
plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.title(f"Detected LEGO Studs ({len(stud_centers)})")
plt.axis("off")
plt.show()

# --- Step 5: Save results ---
os.makedirs('results', exist_ok=True)
cv2.imwrite('results/lego_studs_detected.png', vis)


In [ ]:
# CLUSTER STUDS TO FIND ROI

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import DBSCAN

# --- Step 1: Load image and detect studs (reuse from before) ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

circles = cv2.HoughCircles(
    gray,
    cv2.HOUGH_GRADIENT,
    dp=1.2,
    minDist=25,
    param1=100,
    param2=15,        # inital 20
    minRadius=6,
    maxRadius=15
)

if circles is None:
    raise RuntimeError("No studs detected!")

circles = np.uint16(np.around(circles[0, :]))
stud_points = np.array([[x, y] for x, y, r in circles])

# --- Step 2: Cluster nearby studs using DBSCAN ---
# eps = max distance between studs in a cluster; min_samples = minimum studs per cluster
clustering = DBSCAN(eps=60, min_samples=2).fit(stud_points)
labels = clustering.labels_

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Detected {len(stud_points)} studs and {n_clusters} clusters (potential bricks).")

# --- Step 3: Draw clusters and compute ROIs ---
vis = img.copy()
roi_list = []
for cluster_id in range(n_clusters):
    cluster_points = stud_points[labels == cluster_id]
    if len(cluster_points) < 4:
        continue  # we only want 4-stud bricks

    x_min, y_min = np.min(cluster_points, axis=0)
    x_max, y_max = np.max(cluster_points, axis=0)

    # Slight margin around the studs
    margin = 10
    x1, y1, x2, y2 = x_min - margin, y_min - margin, x_max + margin, y_max + margin
    roi_list.append((int(x1), int(y1), int(x2 - x1), int(y2 - y1)))

    # Draw ROI rectangle
    cv2.rectangle(vis, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
    cv2.putText(vis, f'ROI #{len(roi_list)}', (int(x1), int(y1) - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

# --- Step 4: Display results ---
plt.figure(figsize=(10,8))
plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.title(f'Grouped 4-Stud ROIs ({len(roi_list)} bricks)')
plt.axis('off')
plt.show()

# --- Step 5: Optionally crop and save each ROI ---
os.makedirs('results', exist_ok=True)
for i, (x, y, w, h) in enumerate(roi_list, start=1):
    roi = img[y:y+h, x:x+w]
    cv2.imwrite(f'results/brick_roi_{i}.png', roi)

print(f"Saved {len(roi_list)} ROIs in 'results/' folder.")


## Results - Answer to Question 1a)

Previous code show the creation of ROI, in this case 9 regions of interest were created.

Next cell will treat them to find overlap and eliminate if there exist.

In [ ]:
# REMOVE ROI WITH OVERLAP
# DONE THIS TO HAVE REGIONS THAT CAN BE USED AS ROI TO FIND THE BRICKS

import cv2
import numpy as np

def remove_overlapping_rois(roi_list, iou_threshold=0.3, dist_threshold=0.5):
    """
    Improved version: removes overlapping or nearby ROIs, keeping only the largest.
    - iou_threshold: standard intersection-over-union threshold
    - dist_threshold: normalized center distance threshold (fraction of min dimension)
    """
    def overlap_score(a, b):
        # compute intersection area
        xA = max(a[0], b[0])
        yA = max(a[1], b[1])
        xB = min(a[0]+a[2], b[0]+b[2])
        yB = min(a[1]+a[3], b[1]+b[3])
        interW = max(0, xB - xA)
        interH = max(0, yB - yA)
        interArea = interW * interH
        if interArea == 0:
            return 0.0
        areaA, areaB = a[2]*a[3], b[2]*b[3]
        # relative overlap (more aggressive)
        return interArea / min(areaA, areaB)

    def center_distance(a, b):
        ax, ay = a[0] + a[2]/2, a[1] + a[3]/2
        bx, by = b[0] + b[2]/2, b[1] + b[3]/2
        return np.hypot(ax - bx, ay - by)

    sorted_rois = sorted(roi_list, key=lambda r: r[2]*r[3], reverse=True)
    final_rois = []

    for box in sorted_rois:
        keep = True
        for kept in final_rois:
            overlap = overlap_score(box, kept)
            dist = center_distance(box, kept)
            # compute normalized distance
            min_dim = min(box[2], box[3], kept[2], kept[3])
            if overlap > 0.25 or dist < min_dim * dist_threshold:
                keep = False
                break
        if keep:
            final_rois.append(box)

    return final_rois



# --- Example usage after your DBSCAN grouping code ---
filtered_rois = remove_overlapping_rois(roi_list, iou_threshold=0.3)
print(f"Filtered from {len(roi_list)} → {len(filtered_rois)} ROIs (removed overlaps)")

# --- Visualize filtered ROIs ---
vis_filtered = img.copy()
for i, (x, y, w, h) in enumerate(filtered_rois, start=1):
    cv2.rectangle(vis_filtered, (x, y), (x+w, y+h), (0,255,255), 2)
    cv2.putText(vis_filtered, f"ROI#{i}", (x, y-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)

cv2.imwrite("results/lego_rois_filtered.png", vis_filtered)

print("\n✅ Final non-overlapping ROIs:")
for i, (x, y, w, h) in enumerate(filtered_rois, start=1):
    print(f"ROI #{i}: x={x}, y={y}, w={w}, h={h}, area={w*h}")


In [ ]:
# FIND WHITE BRICKS IN EACH ROI
# ONLY the white are being identified.
# PROOF OF CONCEPT

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

os.makedirs("results/roi_masks", exist_ok=True)

# --- Load original image ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
if img is None:
    raise FileNotFoundError("Image not found: ./data/Isolated/colored_bricks.png")

# 🧩 Example placeholder ROIs (use your real ones if available)
# filtered_rois = [(x, y, w, h), ...]
# If you already have filtered_rois defined, this block will be skipped
if "filtered_rois" not in locals():
    print("⚠️ No filtered_rois found — using demo ROIs")
    h, w = img.shape[:2]
    filtered_rois = [(50, 50, w-100, h-100)]  # one ROI covering most of the image

# 🖼️ Create the global visualization canvas
vis_global = img.copy()

# --- Iterate through each ROI ---
for i, (x, y, w, h) in enumerate(filtered_rois, start=1):
    roi = img[y:y+h, x:x+w]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define white range (fine-tuned for local detection)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 60, 255])
    mask_white = cv2.inRange(hsv, lower_white, upper_white)

    # Morphological cleaning
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel)

    # Find contours of white regions
    contours, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_area = 2000  # adjust this value based on image scale
    white_count = 0
    vis = roi.copy()

    for c in contours:
        area = cv2.contourArea(c)
        if area < min_area:
            continue  # skip tiny false detections

        x2, y2, w2, h2 = cv2.boundingRect(c)
        gx1, gy1 = x + x2, y + y2
        gx2, gy2 = gx1 + w2, gy1 + h2

        # Draw both on the ROI and on the global visualization
        cv2.rectangle(vis, (x2, y2), (x2 + w2, y2 + h2), (180, 180, 180), 2)
        cv2.rectangle(vis_global, (gx1, gy1), (gx2, gy2), (180, 180, 180), 2)
        cv2.putText(vis_global, 'white', (gx1, gy1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 1)
        white_count += 1

    # --- Display / save ---
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(mask_white, cmap='gray')
    plt.title(f"ROI {i} mask")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
    plt.title(f"ROI {i} white bricks: {white_count}")
    plt.axis('off')
    plt.show()

    cv2.imwrite(f"results/roi_masks/roi_{i}_mask.png", mask_white)
    cv2.imwrite(f"results/roi_masks/roi_{i}_detected.png", vis)
    print(f"ROI #{i}: Detected {white_count} white bricks. Saved results.")

# 🖼️ Finally save the global visualization
cv2.imwrite("results/roi_masks/global_white_detections.png", vis_global)
plt.figure(figsize=(10,8))
plt.imshow(cv2.cvtColor(vis_global, cv2.COLOR_BGR2RGB))
plt.title("Global White Brick Detections")
plt.axis("off")
plt.show()


In [ ]:
# DISPLAY IN ORIGINAL PICTURE WHERE THE WHITE BRICKS ARE
# FIND THE WHITE BRICKS
# Only the White are being identified
# PROOF OF CONCEPT

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

os.makedirs("results/final", exist_ok=True)

# --- Step 1: Load original image ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
if img is None:
    raise FileNotFoundError("Image not found")

# Example: your list of filtered, non-overlapping ROIs
# filtered_rois = [(x, y, w, h), ...]

# --- Step 2: Initialize a copy for visualization ---
vis_global = img.copy()
global_white_count = 0

# --- Step 3: Process each ROI ---
for i, (x, y, w, h) in enumerate(filtered_rois, start=1):
    roi = img[y:y+h, x:x+w]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define "white" range
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 60, 255])
    mask_white = cv2.inRange(hsv, lower_white, upper_white)

    # Morphological clean-up
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel)

    # Find contours (white blobs)
    contours, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        area = cv2.contourArea(c)
        if area < 2000:  # small noise filter
            continue
        x2, y2, w2, h2 = cv2.boundingRect(c)

        # Convert local ROI coordinates → global coordinates
        gx1, gy1 = x + x2, y + y2
        gx2, gy2 = gx1 + w2, gy1 + h2

        # Draw rectangle on global image
        cv2.rectangle(vis_global, (gx1, gy1), (gx2, gy2), (180, 180, 180), 2)
        cv2.putText(vis_global, f'white', (gx1, gy1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 1)
        global_white_count += 1

# --- Step 4: Display global results ---
plt.figure(figsize=(10,8))
plt.imshow(cv2.cvtColor(vis_global, cv2.COLOR_BGR2RGB))
plt.title(f"Detected White Bricks (total = {global_white_count})")
plt.axis('off')
plt.show()

# --- Step 5: Save the final annotated image ---
cv2.imwrite("results/final/white_bricks_on_original.png", vis_global)
print(f"✅ Done. Found {global_white_count} white bricks. Saved annotated image to results/final/white_bricks_on_original.png")


In [ ]:
# DISPLAY IN ORIGINAL PICTURE WHERE BRICKS ARE
# FIND ALL BRICKS
# DISPLAY IN ORIGINAL PICTURE
# FINAL RESULT

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

os.makedirs("results/final", exist_ok=True)

# --- Load the original image ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
if img is None:
    raise FileNotFoundError("Image not found")

# --- Assume you already have your final filtered ROIs ---
# filtered_rois = [(x, y, w, h), ...]

# --- Define color ranges in HSV ---
color_ranges = {
    'red1':   [(0, 120, 70), (10, 255, 255)],
    'red2':   [(170, 120, 70), (180, 255, 255)],
    'blue':   [(90, 80, 50), (130, 255, 255)],
    'green':  [(40, 60, 50), (85, 255, 255)],
    'yellow': [(20, 100, 100), (35, 255, 255)],
    'white':  [(0, 0, 200), (180, 60, 255)]
}

# --- Assign BGR colors for drawing ---
color_bgr_map = {
    'red': (0, 0, 255),
    'blue': (255, 0, 0),
    'green': (0, 255, 0),
    'yellow': (0, 255, 255),
    'white': (200, 200, 200)
}

# --- Prepare visualization image ---
vis_global = img.copy()
results = []

# --- Process each ROI ---
for i, (x, y, w, h) in enumerate(filtered_rois, start=1):
    roi = img[y:y+h, x:x+w]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    color_counts = {c: 0 for c in ['red', 'blue', 'green', 'yellow', 'white']}

    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))

        # Merge red1 + red2
        base_color = 'red' if 'red' in color else color

        # Morphological cleanup
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        min_area = 800  # Adjust for your scale
        for c in contours:
            area = cv2.contourArea(c)
            if area < min_area:
                continue
            color_counts[base_color] += 1
            x2, y2, w2, h2 = cv2.boundingRect(c)
            gx1, gy1 = x + x2, y + y2
            gx2, gy2 = gx1 + w2, gy1 + h2
            cv2.rectangle(vis_global, (gx1, gy1), (gx2, gy2), color_bgr_map[base_color], 2)
            cv2.putText(vis_global, base_color, (gx1, gy1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr_map[base_color], 1)

    for c, count in color_counts.items():
        if count > 0:
            results.append({'ROI': i, 'Color': c, 'Count': count})

# --- Convert results to DataFrame ---
df_colors = pd.DataFrame(results)
summary = df_colors.groupby('Color')['Count'].sum().reset_index()

# --- Display summary ---
print("🎨 Summary of bricks by color:")
print(summary)

# --- Save summary ---
summary.to_csv('results/final/brick_color_summary.csv', index=False)

# --- Show final annotated image ---
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(vis_global, cv2.COLOR_BGR2RGB))
plt.title("Detected Bricks by Color")
plt.axis("off")
plt.show()

cv2.imwrite("results/final/bricks_by_color.png", vis_global)
print("✅ Annotated image saved to results/final/bricks_by_color.png")


## Results - Answer to Question 2b)

Previous code show all the bricks being counted and identified in the image colored_bricks.png

Result stored in bricks_by_color.png

#### EXTRA EXPERIMENTS

Tried solution to find all bricks.
Problems with white.

In [ ]:
# APPLY a MASK for each color to ease detection
# TRYING TO DETECT ALL BRICKS
# PLAYING WITH THE SETTINGS OF THE WHITE COLOR
# USING PWHIT TO EXPERIMENT.
# UNSUCCESSFULL

import pandas as pd

def count_colors(img):
    # Convert from BGR (OpenCV default) to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define color ranges in HSV (adjusted for your image)
    color_ranges = {
        'red1':   [(0, 120, 70), (10, 255, 255)],
        'red2':   [(170, 120, 70), (180, 255, 255)],
        'blue':   [(90, 80, 50), (130, 255, 255)],
        'green':  [(40, 60, 50), (85, 255, 255)],
        'yellow': [(20, 100, 100), (35, 255, 255)],
        'white':  [(0, 0, 200), (180, 30, 255)],
        'pwhite': [(0, 0, 200), (30, 50, 255)]
    }

    counts = {}

    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        count = cv2.countNonZero(mask)
        base_color = 'red' if 'red' in color else color
        counts[base_color] = counts.get(base_color, 0) + count
        plt.figure(); plt.imshow(mask,cmap='gray'); plt.title(color)

    return counts


img = cv2.imread('./data/Isolated/colored_bricks.png')
counts = count_colors(img)
df = pd.DataFrame(list(counts.items()), columns=['Color','Count'])

print(df)



In [ ]:
# APPLY a MASK for each color to ease detection
# TRYING TO DETECT ALL BRICKS
# PLAYING WITH THE SETTINGS OF THE WHITE COLOR
# USING PWHIT TO EXPERIMENT.
# UNSUCCESSFULL

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def mask_white(img_bgr):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower_hsv_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_hsv_white = np.array([180, 60, 255], dtype=np.uint8)
    hsv_white = cv2.inRange(hsv, lower_hsv_white, upper_hsv_white)

    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)
    L_min, a_tol, b_tol = 200, 15, 15
    lab_white = (
        (L >= L_min) &
        (cv2.absdiff(A, 128) <= a_tol) &
        (cv2.absdiff(B, 128) <= b_tol)
    ).astype(np.uint8) * 255

    white_mask = cv2.bitwise_or(hsv_white, lab_white)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, k)
    white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, k)
    return white_mask


def count_and_display_bricks(img_bgr, min_area_ratio=0.0002):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    counts = {'red':0,'blue':0,'green':0,'yellow':0,'white':0,'pwhite':0}

    # Colors in HSV space
    color_ranges = {
        'red1':   [(0, 120, 70), (10, 255, 255)],
        'red2':   [(170, 120, 70), (180, 255, 255)],
        'blue':   [(90, 80, 50), (130, 255, 255)],
        'green':  [(40, 60, 50), (85, 255, 255)],
        'yellow': [(20, 100, 100), (35, 255, 255)],
        'pwhite': [(0, 0, 200), (30, 50, 255)]
    }

    # Output copy for visualization
    vis = img_bgr.copy()
    h, w = img_bgr.shape[:2]
    min_area = int(min_area_ratio * h * w)

    color_bgr_map = {
        'red': (0,0,255),
        'blue': (255,0,0),
        'green': (0,255,0),
        'yellow': (0,255,255),
        'white': (200,200,200),
        'pwhite': (200,200,200)
    }

    # Process all non-white colors
    for name, (lo, hi) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lo, np.uint8), np.array(hi, np.uint8))
        base = 'red' if 'red' in name else name
        mask = cv2.medianBlur(mask, 5)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        bricks = [c for c in cnts if cv2.contourArea(c) > min_area]
        counts[base] += len(bricks)
        # Draw each detected brick
        for c in bricks:
            x,y,wc,hc = cv2.boundingRect(c)
            cv2.rectangle(vis, (x,y), (x+wc, y+hc), color_bgr_map[base], 2)
            cv2.putText(vis, base, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr_map[base], 1)

    # Handle white separately
    white_mask = mask_white(img_bgr)
    cnts, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    white_bricks = []
    for c in cnts:
        area = cv2.contourArea(c)
        if area <= min_area:
            continue
        x,y,wc,hc = cv2.boundingRect(c)
        rect_area = wc*hc
        fill_ratio = area/(rect_area+1e-6)
        aspect = max(wc,hc)/(min(wc,hc)+1e-6)
        if fill_ratio>0.65 and aspect<6.0:
            white_bricks.append(c)
            cv2.rectangle(vis, (x,y), (x+wc, y+hc), color_bgr_map['white'], 2)
            cv2.putText(vis, 'white', (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50,50,50), 1)

    counts['white'] = len(white_bricks)

    # Display annotated image
    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,8))
    plt.imshow(vis_rgb)
    plt.title("Detected bricks by color")
    plt.axis("off")
    plt.show()

    return pd.DataFrame(sorted(counts.items()), columns=['Color','Num_Bricks'])


# --- Example usage ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
df = count_and_display_bricks(img)
print(df)


In [ ]:
# APPLY a MASK for each color to ease detection
# TRYING TO DETECT ALL BRICKS
# SPECIAL TREATMENT FOR WHITE
# UNSUCCESSFULL

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def detect_white_bricks_on_white_bg(img_bgr, colored_mask=None, min_area_ratio=0.0002):
    """Detects white bricks using edges + shape analysis, optionally excluding colored regions."""
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    min_area = int(min_area_ratio * h * w)

    # Remove colored areas from search (so we don’t mix)
    if colored_mask is not None:
        mask_inv = cv2.bitwise_not(colored_mask)
    else:
        mask_inv = np.ones_like(gray) * 255

    # Slight blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # Detect edges
    edges = cv2.Canny(blur, 50, 150)

    # Mask to ignore colored zones
    edges = cv2.bitwise_and(edges, edges, mask=mask_inv)

    # Close small gaps in edges
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, k, iterations=1)

    # Find contours
    cnts, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    white_bricks = []
    for c in cnts:
        area = cv2.contourArea(c)
        if area <= min_area:
            continue
        x,y,wc,hc = cv2.boundingRect(c)
        rect_area = wc*hc
        fill_ratio = area/(rect_area+1e-6)
        aspect = max(wc,hc)/(min(wc,hc)+1e-6)
        if fill_ratio>0.65 and aspect<6.0:
            white_bricks.append(c)
    return white_bricks


def count_and_display_bricks(img_bgr):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    h, w = img_bgr.shape[:2]
    min_area = int(0.0002 * h * w)

    color_ranges = {
        'red1':   [(0, 120, 70), (10, 255, 255)],
        'red2':   [(170, 120, 70), (180, 255, 255)],
        'blue':   [(90, 80, 50), (130, 255, 255)],
        'green':  [(40, 60, 50), (85, 255, 255)],
        'yellow': [(20, 100, 100), (35, 255, 255)],
    }

    color_bgr_map = {
        'red': (0,0,255),
        'blue': (255,0,0),
        'green': (0,255,0),
        'yellow': (0,255,255),
        'white': (180,180,180)
    }

    counts = {c:0 for c in color_bgr_map}
    vis = img_bgr.copy()

    # To exclude colored zones later
    combined_color_mask = np.zeros((h,w), np.uint8)

    # Process color bricks
    for name, (lo, hi) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lo, np.uint8), np.array(hi, np.uint8))
        base = 'red' if 'red' in name else name
        combined_color_mask = cv2.bitwise_or(combined_color_mask, mask)
        mask = cv2.medianBlur(mask, 5)
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        bricks = [c for c in cnts if cv2.contourArea(c) > min_area]
        counts[base] += len(bricks)
        for c in bricks:
            x,y,wc,hc = cv2.boundingRect(c)
            cv2.rectangle(vis, (x,y), (x+wc,y+hc), color_bgr_map[base], 2)
            cv2.putText(vis, base, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr_map[base], 1)

    # Detect white bricks using edge analysis
    white_bricks = detect_white_bricks_on_white_bg(img_bgr, combined_color_mask)
    for c in white_bricks:
        x,y,wc,hc = cv2.boundingRect(c)
        cv2.rectangle(vis, (x,y), (x+wc,y+hc), color_bgr_map['white'], 2)
        cv2.putText(vis, 'white', (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50,50,50), 1)
    counts['white'] = len(white_bricks)

    # Show result
    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,8))
    plt.imshow(vis_rgb)
    plt.axis('off')
    plt.title("Detected Bricks (including white on white)")
    plt.show()

    return pd.DataFrame(sorted(counts.items()), columns=['Color','Num_Bricks'])

# --- Example usage ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
df = count_and_display_bricks(img)
print(df)

In [ ]:
# APPLY a MASK for each color to ease detection
# TRYING TO DETECT ALL BRICKS
# SPECIAL TREATMENT FOR WHITE
# UNSUCCESSFULL

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def detect_bricks_all_colors(img_bgr, min_area_ratio=0.0003):
    """
    Detects LEGO bricks by color, including white bricks on a white background.
    Combines color segmentation + edge-based shape detection.
    """

    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    h, w = img_bgr.shape[:2]
    min_area = int(min_area_ratio * h * w)

    # Define HSV color ranges (for colored bricks)
    color_ranges = {
        'red1':   [(0, 120, 70), (10, 255, 255)],
        'red2':   [(170, 120, 70), (180, 255, 255)],
        'blue':   [(90, 80, 50), (130, 255, 255)],
        'green':  [(40, 60, 50), (85, 255, 255)],
        'yellow': [(20, 100, 100), (35, 255, 255)],
    }

    color_bgr_map = {
        'red': (0, 0, 255),
        'blue': (255, 0, 0),
        'green': (0, 255, 0),
        'yellow': (0, 255, 255),
        'white': (180, 180, 180)
    }

    counts = {c: 0 for c in color_bgr_map}
    vis = img_bgr.copy()

    # Create a combined mask to exclude colored bricks from white detection
    combined_mask = np.zeros((h, w), np.uint8)

    # --- Step 1: Detect colored bricks ---
    for name, (lo, hi) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lo, np.uint8), np.array(hi, np.uint8))
        base = 'red' if 'red' in name else name
        combined_mask = cv2.bitwise_or(combined_mask, mask)
        mask = cv2.medianBlur(mask, 5)
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        bricks = [c for c in cnts if cv2.contourArea(c) > min_area]
        counts[base] += len(bricks)
        for c in bricks:
            x, y, wc, hc = cv2.boundingRect(c)
            cv2.rectangle(vis, (x, y), (x + wc, y + hc), color_bgr_map[base], 2)
            cv2.putText(vis, base, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr_map[base], 1)

    # --- Step 2: Detect white bricks using edges ---
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    # Remove colored regions
    search_mask = cv2.bitwise_not(combined_mask)
    gray_masked = cv2.bitwise_and(gray, gray, mask=search_mask)

    # Equalize + blur to enhance contrast
    gray_eq = cv2.equalizeHist(gray_masked)
    blur = cv2.GaussianBlur(gray_eq, (5, 5), 0)

    # Edge detection (low thresholds to pick faint shadows)
    edges = cv2.Canny(blur, 20, 80)

    # Close small gaps in edges
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, k)

    # Find contours (white brick candidates)
    cnts, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    white_bricks = []
    for c in cnts:
        area = cv2.contourArea(c)
        if area <= min_area:
            continue
        x, y, wc, hc = cv2.boundingRect(c)
        rect_area = wc * hc
        fill_ratio = area / (rect_area + 1e-6)
        aspect = max(wc, hc) / (min(wc, hc) + 1e-6)
        # White bricks are rectangular and compact
        if 0.6 < fill_ratio < 1.2 and aspect < 3.0:
            white_bricks.append(c)

    counts['white'] = len(white_bricks)
    for c in white_bricks:
        x, y, wc, hc = cv2.boundingRect(c)
        cv2.rectangle(vis, (x, y), (x + wc, y + hc), color_bgr_map['white'], 2)
        cv2.putText(vis, 'white', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 1)

    # --- Step 3: Display ---
    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(vis_rgb)
    plt.axis('off')
    plt.title('Detected Bricks (all colors, including white)')
    plt.show()

    return pd.DataFrame(sorted(counts.items()), columns=['Color', 'Num_Bricks'])

# --- Example usage ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
df = detect_bricks_all_colors(img)
print(df)

In [ ]:
# APPLY a MASK for each color to ease detection
# TRYING TO DETECT ALL BRICKS
# SPECIAL TREATMENT FOR WHITE
# UNSUCCESSFULL

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def detect_white_bricks_adaptive(img_bgr, exclude_mask=None, min_area_ratio=0.0003):
    """
    Detect white LEGO bricks even on a white background using adaptive thresholding.
    exclude_mask: binary mask of already-detected colored bricks to ignore.
    """
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    min_area = int(min_area_ratio * h * w)

    # Remove colored zones if provided
    if exclude_mask is not None:
        gray = cv2.bitwise_and(gray, gray, mask=cv2.bitwise_not(exclude_mask))

    # Enhance contrast
    gray_eq = cv2.equalizeHist(gray)

    # --- Adaptive threshold: highlight local dark edges of white bricks ---
    th = cv2.adaptiveThreshold(
        gray_eq, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,
        35,   # blockSize (odd) -> adapt to local lighting
        5     # C constant -> adjust sensitivity
    )

    # Clean up noise
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, k, iterations=1)
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN,  k, iterations=1)

    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    white_bricks = []
    for c in cnts:
        area = cv2.contourArea(c)
        if area < min_area:
            continue
        x, y, wc, hc = cv2.boundingRect(c)
        rect_area = wc * hc
        fill_ratio = area / (rect_area + 1e-6)
        aspect = max(wc, hc) / (min(wc, hc) + 1e-6)
        if 0.6 < fill_ratio < 1.1 and 0.7 < aspect < 3.0:
            white_bricks.append(c)

    return white_bricks


def detect_bricks_all_colors(img_bgr):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    h, w = img_bgr.shape[:2]
    min_area = int(0.0003 * h * w)

    color_ranges = {
        'red1':   [(0, 120, 70), (10, 255, 255)],
        'red2':   [(170, 120, 70), (180, 255, 255)],
        'blue':   [(90, 80, 50), (130, 255, 255)],
        'green':  [(40, 60, 50), (85, 255, 255)],
        'yellow': [(20, 100, 100), (35, 255, 255)],
    }

    color_bgr_map = {
        'red': (0, 0, 255),
        'blue': (255, 0, 0),
        'green': (0, 255, 0),
        'yellow': (0, 255, 255),
        'white': (180, 180, 180)
    }

    counts = {c: 0 for c in color_bgr_map}
    vis = img_bgr.copy()
    combined_mask = np.zeros((h, w), np.uint8)

    # --- Colored bricks ---
    for name, (lo, hi) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lo, np.uint8), np.array(hi, np.uint8))
        base = 'red' if 'red' in name else name
        combined_mask = cv2.bitwise_or(combined_mask, mask)
        mask = cv2.medianBlur(mask, 5)
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        bricks = [c for c in cnts if cv2.contourArea(c) > min_area]
        counts[base] += len(bricks)
        for c in bricks:
            x, y, wc, hc = cv2.boundingRect(c)
            cv2.rectangle(vis, (x, y), (x + wc, y + hc), color_bgr_map[base], 2)
            cv2.putText(vis, base, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr_map[base], 1)

    # --- White bricks via adaptive threshold ---
    white_bricks = detect_white_bricks_adaptive(img_bgr, combined_mask)
    counts['white'] = len(white_bricks)
    for c in white_bricks:
        x, y, wc, hc = cv2.boundingRect(c)
        cv2.rectangle(vis, (x, y), (x + wc, y + hc), color_bgr_map['white'], 2)
        cv2.putText(vis, 'white', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 1)

    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(vis_rgb)
    plt.axis('off')
    plt.title('Detected Bricks (including white via adaptive threshold)')
    plt.show()

    return pd.DataFrame(sorted(counts.items()), columns=['Color', 'Num_Bricks'])


# --- Example usage ---
img = cv2.imread('./data/Isolated/colored_bricks.png')
df = detect_bricks_all_colors(img)
print(df)


### Area by brick size

In [ ]:
# PIPELINE TO DETECT BRICK SIZES
# USED STRATEGY FROM BEFORE
# FOR EACH IMAGE CREATED ROI USING THE STUDS AND THEN CLUSTER
# THE DO BRICK DETECTION LOOKING FOR SPECIFIC SIZES (USING THE STUD)
# ALMOST PERFECT - slight error in yellow

import os, glob, shutil
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# ======= CONFIG =======
PX_PER_MM = 0.330046             # mm per pixel
ALLOWED_FILES = {"blue.png", "green.png", "red.png", "yellow.png"}
VALID_STUD_COUNTS = {2: "2x1", 4: "2x2", 8: "4x2", 12: "6x2"}  # exact mapping
STUD_MISS_TOL = 1   # allow ±1 stud and snap to nearest valid count

# ======= UTILS =======
def reset_results():
    base = "results"
    if os.path.exists(base):
        for item in os.listdir(base):
            p = os.path.join(base, item)
            try:
                if os.path.isfile(p) or os.path.islink(p):
                    os.unlink(p)
                else:
                    shutil.rmtree(p)
            except Exception as e:
                print("⚠️ Delete warning:", e)
    os.makedirs("results/annotated", exist_ok=True)
    os.makedirs("results/debug", exist_ok=True)

def mm2_from_px_area(px_area: float) -> float:
    return float(px_area) * (PX_PER_MM ** 2)

def detect_all_studs(img_bgr):
    g = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.equalizeHist(g)
    g = cv2.medianBlur(g, 3)
    # HoughCircles – tune if needed
    circles = cv2.HoughCircles(
        g,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=18,
        param1=90,
        param2=20,            # with 20 it work better than with 15
        minRadius=5,
        maxRadius=16
    )
    if circles is None:
        return np.empty((0, 2), dtype=np.float32)
    C = np.uint16(np.around(circles[0]))
    return C[:, :2].astype(np.float32)   # (x,y)

def estimate_eps_from_spacing(points):
    # Use median nearest-neighbor distance as stud spacing proxy
    if len(points) < 2:
        return 40.0
    from scipy.spatial import cKDTree
    tree = cKDTree(points)
    dists, _ = tree.query(points, k=2)  # nearest neighbor (k=1 is itself)
    nn = np.median(dists[:, 1])
    # eps a bit larger than spacing to collect a brick cluster
    return float(max(1.25 * nn, 25.0))

def boxes_from_stud_clusters(points, labels, margin=10):
    rois = []  # list of (x,y,w,h, cluster_id)
    for lab in sorted(set(labels)):
        if lab == -1:
            continue
        P = points[labels == lab]
        x1, y1 = np.min(P, axis=0)
        x2, y2 = np.max(P, axis=0)
        x1, y1 = int(x1 - margin), int(y1 - margin)
        x2, y2 = int(x2 + margin), int(y2 + margin)
        rois.append((x1, y1, max(1, x2 - x1), max(1, y2 - y1), lab))
    return rois

def remove_overlaps_keep_largest(rois, iou_thresh=0.3, dist_ratio=0.7):
    def inter_over_min(a,b):
        ax1, ay1, aw, ah = a[:4]; ax2, ay2 = ax1+aw, ay1+ah
        bx1, by1, bw, bh = b[:4]; bx2, by2 = bx1+bw, by1+bh
        xA, yA = max(ax1,bx1), max(ay1,by1)
        xB, yB = min(ax2,bx2), min(ay2,by2)
        inter = max(0, xB-xA) * max(0, yB-yA)
        areaA, areaB = aw*ah, bw*bh
        return inter / float(max(1, min(areaA, areaB)))

    def center_dist(a,b):
        ax = a[0] + a[2]/2; ay = a[1] + a[3]/2
        bx = b[0] + b[2]/2; by = b[1] + b[3]/2
        return np.hypot(ax-bx, ay-by)

    rois_sorted = sorted(rois, key=lambda r: r[2]*r[3], reverse=True)
    keep = []
    for r in rois_sorted:
        ok = True
        for k in keep:
            ov = inter_over_min(r,k)
            mind = min(r[2], r[3], k[2], k[3])
            if ov > 0.25 or center_dist(r,k) < mind * dist_ratio:
                ok = False; break
        if ok: keep.append(r)
    return keep

def snap_stud_count(n):
    # snap n to nearest in {2,4,8,12} if within ±STUD_MISS_TOL; else return None
    candidates = np.array(sorted(VALID_STUD_COUNTS.keys()))
    idx = np.argmin(np.abs(candidates - n))
    nearest = int(candidates[idx])
    return nearest if abs(nearest - n) <= STUD_MISS_TOL else None

def measure_brick_area_in_roi(img_bgr, roi):
    x,y,w,h = roi[:4]
    crop = img_bgr[max(0,y):y+h, max(0,x):x+w]
    if crop.size == 0: return None, None
    g = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    # Binary (brick brighter vs background in isolated images). Invert to white-blob.
    _, th = cv2.threshold(g, 127, 255, cv2.THRESH_BINARY_INV)
    # Fill stud holes so area is footprint
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT,(15,15)))
    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return None, th
    c = max(cnts, key=cv2.contourArea)
    return float(cv2.contourArea(c)), th

# ======= MAIN PIPELINE =======
def process_folder(folder):
    reset_results()
    per_file_tables = []
    all_bricks = []

    for path in glob.glob(os.path.join(folder, "*.png")):
        file = os.path.basename(path)
        if file not in ALLOWED_FILES:
            continue

        img = cv2.imread(path)
        if img is None:
            print("⚠️ Cannot read:", file); continue

        # 1) Detect studs globally
        studs = detect_all_studs(img)

        # If none, skip gracefully
        if len(studs) == 0:
            print(f"⚠️ No studs detected in {file}")
            continue

        # 2) Cluster studs -> per-brick ROIs
        eps = estimate_eps_from_spacing(studs)
        labels = DBSCAN(eps=eps, min_samples=2).fit_predict(studs)
        rois = boxes_from_stud_clusters(studs, labels, margin=12)
        rois = remove_overlaps_keep_largest(rois, iou_thresh=0.3, dist_ratio=0.7)

        # 3) For each ROI: count studs (cluster members), classify size, measure area
        vis = img.copy()
        bricks_rows = []

        for (x,y,w,h, lab) in rois:
            # studs that belong to this cluster id
            studs_in = studs[labels == lab]
            n_studs = len(studs_in)

            # snap stud count to a valid brick type (2/4/8/12) with tolerance
            snapped = snap_stud_count(n_studs)
            if snapped is None:
                # try a quick local re-detect inside ROI if count way off
                crop = img[max(0,y):y+h, max(0,x):x+w]
                studs_local = detect_all_studs(crop)
                n_studs = len(studs_local)
                snapped = snap_stud_count(n_studs)

            if snapped is None:
                # still unknown → skip (or label unknown)
                label = "unknown"
            else:
                label = VALID_STUD_COUNTS[snapped]

            # measure area from footprint inside ROI
            area_px, th = measure_brick_area_in_roi(img, (x,y,w,h))
            if area_px is None:
                continue
            area_mm = mm2_from_px_area(area_px)

            # draw annotation
            cv2.rectangle(vis, (x,y), (x+w,y+h), (0,255,255), 2)
            cv2.putText(vis, f"{label}", (x, y-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
            # draw studs (centers) for this brick
            for (sx, sy) in studs[labels == lab]:
                cv2.circle(vis, (int(sx), int(sy)), 3, (255,0,0), -1)

            bricks_rows.append({
                "File": file,
                "Size": label,
                "Studs": n_studs,
                "Area (pixel)": area_px,
                "Area (mm²)": area_mm
            })
            all_bricks.append(bricks_rows[-1])

        # Save annotated image
        cv2.imwrite(os.path.join("results/annotated", file), vis)

        # 4) Per-file table by size
        if bricks_rows:
            df_b = pd.DataFrame(bricks_rows)
            # keep only valid sizes
            df_b = df_b[df_b["Size"].isin(VALID_STUD_COUNTS.values())]
            if not df_b.empty:
                agg = (df_b.groupby("Size")
                        .agg(**{
                            "Number of Bricks": ("Area (pixel)", "count"),
                            "Avg Area (pixel)": ("Area (pixel)", "mean"),
                            "Avg Area (mm²)": ("Area (mm²)", "mean"),
                            "Standard deviation (mm²)": ("Area (mm²)", "std"),
                        })
                        .reset_index())
                agg.insert(0, "File", file)
                per_file_tables.append(agg)

        # Optional quick view
        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title(f"{file} — bricks found & labeled")
        plt.axis("off")
        plt.show()

    # 5) Output tables
    # Per-image
    if per_file_tables:
        df_files = pd.concat(per_file_tables, ignore_index=True)
    else:
        df_files = pd.DataFrame(columns=["File","Size","Number of Bricks","Avg Area (pixel)","Avg Area (mm²)","Standard deviation (mm²)"])

    print("\n📋 Table — Per Image (by Size)")
    display(df_files)
    df_files.to_csv("results/per_image_table.csv", index=False)

    # Global summary
    if all_bricks:
        df_all = pd.DataFrame(all_bricks)
        df_all = df_all[df_all["Size"].isin(VALID_STUD_COUNTS.values())]
        if not df_all.empty:
            df_summary = (df_all.groupby("Size")
                          .agg(**{
                              "Number of Bricks": ("Area (pixel)", "count"),
                              "Avg Area (pixel)": ("Area (pixel)", "mean"),
                              "Avg Area (mm²)": ("Area (mm²)", "mean"),
                              "Standard deviation (mm²)": ("Area (mm²)", "std"),
                          })
                          .reset_index())
        else:
            df_summary = pd.DataFrame(columns=["Size","Number of Bricks","Avg Area (pixel)","Avg Area (mm²)","Standard deviation (mm²)"])
    else:
        df_summary = pd.DataFrame(columns=["Size","Number of Bricks","Avg Area (pixel)","Avg Area (mm²)","Standard deviation (mm²)"])

    print("\n📊 Table — Resume (all images)")
    display(df_summary)
    df_summary.to_csv("results/resume_table.csv", index=False)

    print("\n✅ Saved:")
    print(" - results/annotated/<file>.png")
    print(" - results/per_image_table.csv")
    print(" - results/resume_table.csv")

# ======= RUN =======
process_folder("./data/Isolated")


In [ ]:
# PIPELINE TO DETECT BRICK SIZES
# Final refined version: with ROI debug visualization, adjustable EPS + ROI margin, and noise/overlap fixes
# Add MAKS method to help identify good studs.
# FINAL VERSION

import os, glob, shutil
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial import cKDTree

# ======= CONFIG =======
PX_PER_MM = 0.330046
ALLOWED_FILES = {"blue.png", "green.png", "red.png", "yellow.png"}
VALID_STUD_COUNTS = {2: "2x1", 4: "2x2", 8: "4x2", 12: "6x2"}
STUD_MISS_TOL = 1

# 🔧 Adjustable parameters
EPS_MULTIPLIER = 1.23   # controls clustering sensitivity (1.10 = more split, 1.25 = more merge)
# 1.23 reveal to be better option to split the ROI

ROI_MARGIN = 10         # padding (px) added around each ROI

# ======= UTILS =======
def reset_results():
    base = "results"
    if os.path.exists(base):
        shutil.rmtree(base)
    os.makedirs("results/annotated", exist_ok=True)
    os.makedirs("results/debug", exist_ok=True)

def mm2_from_px_area(px_area):
    return float(px_area) * (PX_PER_MM ** 2)

def detect_all_studs(img_bgr):
    """Detect LEGO studs using tuned HoughCircles."""
    g = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.equalizeHist(g)
    g = cv2.medianBlur(g, 3)
    circles = cv2.HoughCircles(
        g, cv2.HOUGH_GRADIENT, dp=1.2, minDist=18,
        param1=90, param2=20, minRadius=5, maxRadius=16
    )
    if circles is None:
        return np.empty((0, 2), dtype=np.float32)
    C = np.uint16(np.around(circles[0]))
    return C[:, :2].astype(np.float32)

def filter_isolated_studs(studs, max_nn_ratio=1.8):
    """Remove isolated studs far from their nearest neighbor."""
    if len(studs) < 2:
        return studs
    tree = cKDTree(studs)
    dists, _ = tree.query(studs, k=2)
    nn = dists[:, 1]
    median_nn = np.median(nn)
    mask = nn < (max_nn_ratio * median_nn)
    return studs[mask]

def estimate_eps_from_spacing(points):
    """Estimate DBSCAN eps from stud spacing."""
    if len(points) < 2:
        return 40.0
    tree = cKDTree(points)
    dists, _ = tree.query(points, k=2)
    nn = np.median(dists[:, 1])
    return float(max(EPS_MULTIPLIER * nn, 25.0))

def boxes_from_stud_clusters(points, labels, margin=ROI_MARGIN):
    """Create ROIs (bounding boxes) from clustered studs."""
    rois = []
    for lab in sorted(set(labels)):
        if lab == -1: continue
        P = points[labels == lab]
        x1, y1 = np.min(P, axis=0)
        x2, y2 = np.max(P, axis=0)
        x1, y1 = int(x1 - margin), int(y1 - margin)
        x2, y2 = int(x2 + margin), int(y2 + margin)
        rois.append((x1, y1, max(1, x2 - x1), max(1, y2 - y1), lab))
    return rois

def remove_overlaps_keep_largest(rois, iou_thresh=0.4, dist_ratio=0.6):
    """Gentle overlap filtering to preserve small bricks."""
    def inter_over_min(a,b):
        ax1,ay1,aw,ah=a[:4]; bx1,by1,bw,bh=b[:4]
        ax2,ay2=ax1+aw,ay1+ah; bx2,by2=bx1+bw,by1+bh
        xA,yA=max(ax1,bx1),max(ay1,by1)
        xB,yB=min(ax2,bx2),min(ay2,by2)
        inter=max(0,xB-xA)*max(0,yB-yA)
        areaA,areaB=aw*ah,bw*bh
        return inter/max(1,min(areaA,areaB))
    def center_dist(a,b):
        ax,ay=a[0]+a[2]/2,a[1]+a[3]/2
        bx,by=b[0]+b[2]/2,b[1]+b[3]/2
        return np.hypot(ax-bx,ay-by)
    keep=[]
    for r in sorted(rois,key=lambda x:x[2]*x[3],reverse=True):
        if all(not (inter_over_min(r,k)>iou_thresh and center_dist(r,k)<min(r[2],r[3],k[2],k[3])*dist_ratio) for k in keep):
            keep.append(r)
    return keep
'''
def snap_stud_count(n):
    """Snap stud count to valid size (2,4,8,12)."""
    candidates = np.array(sorted(VALID_STUD_COUNTS.keys()))
    idx = np.argmin(np.abs(candidates - n))
    nearest = int(candidates[idx])
    return nearest if abs(nearest - n) <= STUD_MISS_TOL else None
'''

def snap_stud_count(n):
    """
    Snap stud count to nearest valid size (2, 4, 8, 12)
    with ±STUD_MISS_TOL tolerance, preferring upward snaps on ties.
    """
    n = int(n)
    candidates = np.array(sorted(VALID_STUD_COUNTS.keys()))  # [2, 4, 8, 12]
    diffs = np.abs(candidates - n)
    min_diff = diffs.min()

    # candidates with same min difference
    nearest_candidates = candidates[diffs == min_diff]
    # prefer larger when tied
    nearest = int(nearest_candidates.max())

    if abs(nearest - n) <= STUD_MISS_TOL:
        return nearest
    return None

def measure_brick_area_in_roi(img_bgr, roi):
    """Measure the area of the brick footprint inside an ROI."""
    x,y,w,h = roi[:4]
    crop = img_bgr[max(0,y):y+h, max(0,x):x+w]
    if crop.size == 0:
        return None, None
    g = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    _, th = cv2.threshold(g, 127, 255, cv2.THRESH_BINARY_INV)
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT,(15,15)))
    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None, th
    c = max(cnts, key=cv2.contourArea)
    return float(cv2.contourArea(c)), th

# NEW FUNCTION THAT I SUGEST TO ADD
def remove_too_close_studs(studs, min_dist=8):
    """
    Removes duplicate or overlapping stud detections.
    Keeps only one stud from each pair that are closer than min_dist pixels.
    """
    if len(studs) < 3:
        return studs
    keep = []
    used = np.zeros(len(studs), dtype=bool)
    for i in range(len(studs)):
        if used[i]:
            continue
        keep.append(studs[i])
        for j in range(i + 1, len(studs)):
            if np.linalg.norm(studs[i] - studs[j]) < min_dist:
                used[j] = True
    return np.array(keep, dtype=np.float32)

# ANOTHER NEW FUNCTION

def keep_colored_studs(img_bgr, studs, sat_thresh=60, val_min=50, val_max=240):
    """
    Keeps only studs located over sufficiently colored (non-white) pixels.
    Filters out studs over low-saturation or near-white background areas.
    """
    if len(studs) == 0:
        return studs
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    kept = []
    for (x, y) in studs:
        x = int(round(x))
        y = int(round(y))
        if y < 0 or y >= hsv.shape[0] or x < 0 or x >= hsv.shape[1]:
            continue
        h, s, v = hsv[y, x]
        # keep studs with decent saturation and not too bright
        if s > sat_thresh and val_min < v < val_max:
            kept.append([x, y])
    return np.array(kept, dtype=np.float32)

def studs_inside_edges(img_bgr, studs, roi, canny1=70, canny2=150):
    """
    Returns only the studs whose centers fall inside the main edge contour
    of the brick within the given ROI.
    """
    x, y, w, h = roi[:4]
    crop = img_bgr[max(0,y):y+h, max(0,x):x+w]
    if crop.size == 0 or len(studs) == 0:
        return studs

    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    #edges = cv2.Canny(gray, canny1, canny2)

    # compute median intensity in ROI
    med_val = np.median(gray)
    sigma = 0.33
    low = int(max(0, (1.0 - sigma) * med_val))
    high = int(min(255, (1.0 + sigma) * med_val))
    edges = cv2.Canny(gray, low, high)

    cnts, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return studs

    # Take the largest contour as the brick boundary
    c = max(cnts, key=cv2.contourArea)

    # Convert studs from global to local ROI coordinates
    studs_local = studs - np.array([x, y], dtype=np.float32)

    kept = []
    for (sx, sy) in studs_local:
        inside = cv2.pointPolygonTest(c, (float(sx), float(sy)), measureDist=False)
        if inside >= 0:   # >=0 means point inside or on edge
            kept.append([sx + x, sy + y])  # convert back to global coords
    return np.array(kept, dtype=np.float32)

def make_color_mask(img_bgr, color):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    if color == 'yellow':
        lower, upper = (10, 100, 100), (35, 255, 255)
    elif color == 'red':
        lower1, upper1 = (0, 120, 70), (10, 255, 255)
        lower2, upper2 = (170, 120, 70), (180, 255, 255)
        mask = cv2.inRange(hsv, np.array(lower1), np.array(upper1)) | cv2.inRange(hsv, np.array(lower2), np.array(upper2))
        return mask
    elif color == 'blue':
        lower, upper = (90, 80, 50), (130, 255, 255)
    elif color == 'green':
        lower, upper = (40, 60, 50), (85, 255, 255)
    else:
        raise ValueError(f"Unknown color: {color}")

    mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((9,9), np.uint8))
    return mask

def keep_studs_inside_mask(studs, mask):
    h, w = mask.shape
    kept = []
    for (x, y) in studs:
        xi, yi = int(round(x)), int(round(y))
        if 0 <= xi < w and 0 <= yi < h and mask[yi, xi] > 0:
            kept.append([x, y])
    return np.array(kept, dtype=np.float32)


# ======= MAIN PIPELINE =======
def process_folder(folder):
    reset_results()
    per_file_tables, all_bricks = [], []

    for path in glob.glob(os.path.join(folder, "*.png")):
        file = os.path.basename(path)
        if file not in ALLOWED_FILES:
            continue
        img = cv2.imread(path)
        if img is None:
            continue

        # Extract color name from file (e.g., "blue.png" → "blue")
        color_name = os.path.splitext(file)[0].lower()

        # --- Detect & clean studs ---

        studs = detect_all_studs(img)

        # Remove studs over non-colored (white/gray) areas
        #studs = keep_colored_studs(img, studs, sat_thresh=40, val_min=40, val_max=250)
        # commented: this is no longer necessary - using the mask make the removal of the noise studs redundant

        # Build color mask for this brick color
        mask = make_color_mask(img, color=color_name)

        # Keep only studs inside mask region
        studs = keep_studs_inside_mask(studs, mask)

        # Optional: also remove near-duplicate studs
        studs = remove_too_close_studs(studs, min_dist=30)
        # This is important to correct classification - help removing noisy studs

        # Remove very isolated ones
        #studs = filter_isolated_studs(studs)
        # commented: this is no longer necessary - using the mask make the removal of the noise studs redundant

        if len(studs) == 0:
            print(f"⚠️ No studs in {file}")
            continue

        # --- Cluster studs ---
        eps = estimate_eps_from_spacing(studs)
        labels = DBSCAN(eps=eps, min_samples=2).fit_predict(studs)
        rois = boxes_from_stud_clusters(studs, labels)
        rois = remove_overlaps_keep_largest(rois)

        # --- ROI DEBUG VISUALIZATION ---
        dbg = img.copy()
        for i, (x,y,w,h,lab) in enumerate(rois, start=1):
            cv2.rectangle(dbg, (x,y), (x+w,y+h), (0,0,255), 2)
            cv2.putText(dbg, f"ROI {i}", (x, y-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
        for (sx,sy) in studs:
            cv2.circle(dbg, (int(sx), int(sy)), 3, (255,0,0), -1)
        cv2.imwrite(f"results/debug/{os.path.splitext(file)[0]}_rois.png", dbg)
        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(dbg, cv2.COLOR_BGR2RGB))
        plt.title(f"{file} — ROI Debug (red boxes = clusters, blue dots = studs)")
        plt.axis("off")
        plt.show()

        # --- Brick Classification and Area ---
        vis = img.copy()
        bricks_rows = []

        for (x,y,w,h,lab) in rois:
            studs_in = studs[labels == lab]

            # ADD NEW CODE - edges
            #studs_in = studs_inside_edges(img, studs_in, (x, y, w, h))
            # END OF ADD CODE

            n_studs = len(studs_in)
            snapped = snap_stud_count(n_studs)

            if snapped is None:
                crop = img[max(0,y):y+h, max(0,x):x+w]
                studs_local = detect_all_studs(crop)
                studs_local = filter_isolated_studs(studs_local)
                n_studs = len(studs_local)
                snapped = snap_stud_count(n_studs)

            label = VALID_STUD_COUNTS.get(snapped, "unknown")
            area_px, th = measure_brick_area_in_roi(img, (x,y,w,h))
            if area_px is None:
                continue
            area_mm = mm2_from_px_area(area_px)

            cv2.rectangle(vis, (x,y), (x+w,y+h), (0,255,255), 2)
            cv2.putText(vis, label, (x, y-6),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
            for (sx,sy) in studs_in:
                cv2.circle(vis, (int(sx), int(sy)), 3, (255,0,0), -1)

            bricks_rows.append({
                "File": file,
                "Size": label,
                "Studs": n_studs,
                "Area (pixel)": area_px,
                "Area (mm²)": area_mm
            })
            all_bricks.append(bricks_rows[-1])

        cv2.imwrite(os.path.join("results/annotated", file), vis)
        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title(f"{file} — Bricks Labeled (yellow boxes)")
        plt.axis("off")
        plt.show()

        # --- Per-file Summary ---
        if bricks_rows:
            df_b = pd.DataFrame(bricks_rows)
            df_b = df_b[df_b["Size"].isin(VALID_STUD_COUNTS.values())]
            if not df_b.empty:
                agg = (df_b.groupby("Size")
                        .agg(**{
                            "Number of Bricks": ("Area (pixel)", "count"),
                            "Avg Area (pixel)": ("Area (pixel)", "mean"),
                            "Avg Area (mm²)": ("Area (mm²)", "mean"),
                            "Standard deviation (mm²)": ("Area (mm²)", "std")
                        })
                        .reset_index())
                agg.insert(0, "File", file)
                per_file_tables.append(agg)

    # --- Tables ---
    df_files = pd.concat(per_file_tables, ignore_index=True) if per_file_tables else pd.DataFrame()
    print("\n📋 Table — Per Image (by Size)")
    display(df_files)
    df_files.to_csv("results/per_image_table.csv", index=False)

    if all_bricks:
        df_all = pd.DataFrame(all_bricks)
        df_all = df_all[df_all["Size"].isin(VALID_STUD_COUNTS.values())]
        df_summary = (df_all.groupby("Size")
                      .agg(**{
                          "Number of Bricks": ("Area (pixel)", "count"),
                          "Avg Area (pixel)": ("Area (pixel)", "mean"),
                          "Avg Area (mm²)": ("Area (mm²)", "mean"),
                          "Standard deviation (mm²)": ("Area (mm²)", "std")
                      })
                      .reset_index())
    else:
        df_summary = pd.DataFrame()

    print("\n📊 Table — Resume (all images)")
    display(df_summary)
    df_summary.to_csv("results/resume_table.csv", index=False)
    print(f"\n✅ Done with EPS_MULTIPLIER={EPS_MULTIPLIER}, ROI_MARGIN={ROI_MARGIN}")
    print("   Results saved to results/annotated/ and results/debug/")

# ======= RUN =======
process_folder("./data/Isolated")


## Results - Answer to Question 2c)

Previous code show all the bricks being counted by format.


## 3️⃣ Kit Images

### Count bricks by color per kit (kit 1, 2, 3)

In [ ]:
# LEGO Kit Brick Detection (Multi-color version)
# Based on your final refined "Isolated" pipeline — adapted for Kit images

import os, glob, shutil, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial import cKDTree

# ======= CONFIG =======
PX_PER_MM = 0.330046
VALID_STUD_COUNTS = {2: "2x1", 4: "2x2", 8: "4x2", 12: "6x2"}
STUD_MISS_TOL = 1
EPS_MULTIPLIER = 1.23
ROI_MARGIN = 10
COLORS = ["red", "blue", "green", "yellow"]

# ======= UTILS =======
def reset_results():
    base = "results"
    if os.path.exists(base):
        shutil.rmtree(base)
    os.makedirs("results/annotated", exist_ok=True)
    os.makedirs("results/debug", exist_ok=True)

def mm2_from_px_area(px_area):
    return float(px_area) * (PX_PER_MM ** 2)
'''
def detect_all_studs(img_bgr):
    g = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.equalizeHist(g)
    g = cv2.medianBlur(g, 3)
    circles = cv2.HoughCircles(
        g, cv2.HOUGH_GRADIENT, dp=1.2, minDist=18,
        param1=90, param2=20, minRadius=5, maxRadius=16
    )
    if circles is None:
        return np.empty((0, 2), dtype=np.float32)
    C = np.uint16(np.around(circles[0]))
    return C[:, :2].astype(np.float32)
'''

def detect_all_studs(img_bgr, color_hint=None):
    """
    Detect LEGO studs using tuned HoughCircles, adaptive per color_hint.
    color_hint can be one of: 'red', 'yellow', 'green', 'blue', 'white', or None.
    """

    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    gray = cv2.medianBlur(gray, 3)

    # --- Default parameters ---
    param1 = 90   # upper threshold for edge detection
    param2 = 20   # accumulator threshold (lower → more circles)
    minR, maxR = 5, 16

    # --- Adjust based on color ---
    if color_hint == 'red':
        param2 = 17   # more sensitive
    elif color_hint == 'yellow':
        param2 = 18   # stricter (avoid false positives) 23
        param1 = 80   # was 100 com 80 melhorou
    elif color_hint == 'green':
        param2 = 20
    elif color_hint == 'blue':
        param2 = 20
    elif color_hint == 'white':
        param2 = 22

    circles = cv2.HoughCircles(
        gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=18,
        param1=param1, param2=param2,
        minRadius=minR, maxRadius=maxR
    )

    if circles is None:
        return np.empty((0, 2), dtype=np.float32)

    C = np.uint16(np.around(circles[0]))
    return C[:, :2].astype(np.float32)

def remove_too_close_studs(studs, min_dist=8):
    if len(studs) < 3: return studs
    keep = []
    used = np.zeros(len(studs), dtype=bool)
    for i in range(len(studs)):
        if used[i]: continue
        keep.append(studs[i])
        for j in range(i+1, len(studs)):
            if np.linalg.norm(studs[i]-studs[j]) < min_dist:
                used[j] = True
    return np.array(keep, dtype=np.float32)

def estimate_eps_from_spacing(points):
    if len(points) < 2: return 40.0
    tree = cKDTree(points)
    dists, _ = tree.query(points, k=2)
    nn = np.median(dists[:, 1])
    return float(max(EPS_MULTIPLIER * nn, 25.0))

def boxes_from_stud_clusters(points, labels, margin=ROI_MARGIN):
    rois = []
    for lab in sorted(set(labels)):
        if lab == -1: continue
        P = points[labels == lab]
        x1, y1 = np.min(P, axis=0)
        x2, y2 = np.max(P, axis=0)
        x1, y1 = int(x1 - margin), int(y1 - margin)
        x2, y2 = int(x2 + margin), int(y2 + margin)
        rois.append((x1, y1, max(1,x2-x1), max(1,y2-y1), lab))
    return rois

def remove_overlaps_keep_largest(rois, iou_thresh=0.4, dist_ratio=0.6):
    """Gentle overlap filtering to preserve small bricks."""
    def inter_over_min(a, b):
        ax1, ay1, aw, ah = a[:4]
        bx1, by1, bw, bh = b[:4]
        ax2, ay2 = ax1 + aw, ay1 + ah
        bx2, by2 = bx1 + bw, by1 + bh
        xA, yA = max(ax1, bx1), max(ay1, by1)
        xB, yB = min(ax2, bx2), min(ay2, by2)
        inter = max(0, xB - xA) * max(0, yB - yA)
        areaA, areaB = aw * ah, bw * bh
        return inter / max(1, min(areaA, areaB))

    keep = []
    # Sort by area descending — keep bigger first
    for r in sorted(rois, key=lambda x: x[2] * x[3], reverse=True):
        if all(inter_over_min(r, k) < iou_thresh for k in keep):
            keep.append(r)
    return keep
'''
def snap_stud_count(n):
    candidates = np.array(sorted(VALID_STUD_COUNTS.keys()))
    idx = np.argmin(np.abs(candidates - n))
    nearest = int(candidates[idx])
    return nearest if abs(nearest - n) <= STUD_MISS_TOL else None
'''
def snap_stud_count(n):
    """
    Snap stud count to nearest valid size (2, 4, 8, 12)
    with ±STUD_MISS_TOL tolerance, preferring upward snaps on ties.
    """
    n = int(n)
    candidates = np.array(sorted(VALID_STUD_COUNTS.keys()))  # [2, 4, 8, 12]
    diffs = np.abs(candidates - n)
    min_diff = diffs.min()

    # candidates with same min difference
    nearest_candidates = candidates[diffs == min_diff]
    # prefer larger when tied
    nearest = int(nearest_candidates.max())

    if abs(nearest - n) <= STUD_MISS_TOL:
        return nearest
    return None


def make_color_mask(img_bgr, color):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    if color == "yellow":
        lower, upper = (10, 100, 100), (35, 255, 255)
    elif color == "red":
        lower1, upper1 = (0, 120, 70), (10, 255, 255)
        lower2, upper2 = (170, 120, 70), (180, 255, 255)
        return (cv2.inRange(hsv, np.array(lower1), np.array(upper1)) |
                cv2.inRange(hsv, np.array(lower2), np.array(upper2)))
    elif color == "blue":
        lower, upper = (90, 80, 50), (130, 255, 255)
    elif color == "green":
        lower, upper = (40, 60, 50), (85, 255, 255)
    else:
        raise ValueError(f"Unknown color: {color}")
    mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((9,9), np.uint8))
    return mask

def keep_studs_inside_mask(studs, mask):
    h, w = mask.shape
    kept = []
    for (x, y) in studs:
        xi, yi = int(round(x)), int(round(y))
        if 0 <= xi < w and 0 <= yi < h and mask[yi, xi] > 0:
            kept.append([x, y])
    return np.array(kept, dtype=np.float32)

# ======= MAIN =======

def process_kits(folder="./data/Kit"):

    reset_results()
    all_rows = []
    for kit_path in sorted(glob.glob(os.path.join(folder, "kit*.png"))):

        name = os.path.basename(kit_path)
        print(f"\n🔹 Processing {name}")
        img = cv2.imread(kit_path)
        dbg, vis = img.copy(), img.copy()
        roi_id = 1

        for color in COLORS:
            mask = make_color_mask(img, color)
            #studs = detect_all_studs(img)
            studs = detect_all_studs(img, color_hint=color)
            studs = keep_studs_inside_mask(studs, mask)
            studs = remove_too_close_studs(studs, min_dist=30)
            if len(studs)==0:
                continue

            eps = estimate_eps_from_spacing(studs)
            labels = DBSCAN(eps=eps, min_samples=2).fit_predict(studs)
            rois = boxes_from_stud_clusters(studs, labels)
            rois = remove_overlaps_keep_largest(rois)

            for (x,y,w,h,lab) in rois:
                studs_in = studs[labels==lab]
                n_studs = len(studs_in)
                snapped = snap_stud_count(n_studs)
                label = VALID_STUD_COUNTS.get(snapped, "unknown")

                cv2.rectangle(dbg,(x,y),(x+w,y+h),(0,0,255),2)
                cv2.putText(dbg,f"ROI {roi_id}",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)
                for (sx,sy) in studs_in:
                    cv2.circle(dbg,(int(sx),int(sy)),3,(255,0,0),-1)
                roi_id+=1

                cv2.rectangle(vis,(x,y),(x+w,y+h),(0,255,255),2)
                cv2.putText(vis,f"{color} {label}",(x,y-5),
                            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,255),2)

                # --- Draw centroid in black ---
                cx, cy = int(x + w / 2), int(y + h / 2)
                cv2.circle(dbg, (cx, cy), 4, (0, 0, 0), -1)
                cv2.circle(vis, (cx, cy), 4, (0, 0, 0), -1)


                all_rows.append({"Kit":name,"Color":color,"Size":label,"Studs":n_studs})

        # Save and show debug views
        cv2.imwrite(f"results/debug/{name}_rois.png", dbg)
        cv2.imwrite(f"results/annotated/{name}_bricks.png", vis)
        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(dbg, cv2.COLOR_BGR2RGB))
        plt.title(f"{name} — ROI Debug (red boxes = clusters, blue dots = studs)")
        plt.axis("off")
        plt.show()
        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title(f"{name} — Bricks Labeled (yellow boxes)")
        plt.axis("off")
        plt.show()


    # ===== Pretty summary print: kit name & format shown only once per group =====
    if all_rows:
        df = pd.DataFrame(all_rows)

        # Aggregate results
        summary = (
            df.groupby(["Kit", "Size", "Color"])
              .size()
              .reset_index(name="quantity")
              .sort_values(["Kit", "Size", "Color"])
        )

        print("\n📊 Final Results — Bricks per Kit\n")

        pretty_blocks = []
        for kit_name, sub in summary.groupby("Kit", sort=False):
            sub = sub.copy()
            # Rename columns only once
            sub = sub.rename(columns={"Kit": "kit name", "Size": "format", "Color": "color"})

            # Make 'kit name' column blank except for first row
            sub.loc[sub.index[0], "kit name"] = kit_name
            sub.loc[sub.index[1]:, "kit name"] = ""

            # Remove repeated formats within the same kit
            last_fmt = None
            for i in sub.index:
                fmt = sub.at[i, "format"]
                if fmt == last_fmt:
                    sub.at[i, "format"] = ""
                else:
                    last_fmt = fmt

            pretty_blocks.append(sub[["kit name", "format", "color", "quantity"]])

        pretty_summary = pd.concat(pretty_blocks, ignore_index=True)

        # Display neatly without index column
        display(pretty_summary.style.hide(axis="index"))

        # Save full detailed CSV (with all values)
        summary.to_csv("results/kit_summary.csv", index=False)
        print("✅ Summary saved to results/kit_summary.csv")
    else:
        print("No results to summarize.")

# ======= RUN =======
process_kits()


## Results - Answer to Question 3a) and 3b)

Previous code show for each kit the quantity by format and color.
Displays the detection of each brick with the centroid for each brick in black.

In [ ]:
# PROCESS ALL KIT IN FOLDER
# SEPARATE REFERENCE KITS FROM TESTING ONES
# EVALUATE SIMILARITY

def process_all_kits(folder="./data/Kit"):

    reset_results()
    all_rows = []
    for kit_path in sorted(glob.glob(os.path.join(folder, "*.png"))):

        name = os.path.basename(kit_path)
        print(f"\n🔹 Processing {name}")
        img = cv2.imread(kit_path)
        dbg, vis = img.copy(), img.copy()
        roi_id = 1

        for color in COLORS:
            mask = make_color_mask(img, color)
            #studs = detect_all_studs(img)
            studs = detect_all_studs(img, color_hint=color)
            studs = keep_studs_inside_mask(studs, mask)
            studs = remove_too_close_studs(studs, min_dist=30)
            if len(studs)==0:
                continue

            eps = estimate_eps_from_spacing(studs)
            labels = DBSCAN(eps=eps, min_samples=2).fit_predict(studs)
            rois = boxes_from_stud_clusters(studs, labels)
            rois = remove_overlaps_keep_largest(rois)

            for (x,y,w,h,lab) in rois:
                studs_in = studs[labels==lab]
                n_studs = len(studs_in)
                snapped = snap_stud_count(n_studs)
                label = VALID_STUD_COUNTS.get(snapped, "unknown")

                cv2.rectangle(dbg,(x,y),(x+w,y+h),(0,0,255),2)
                cv2.putText(dbg,f"ROI {roi_id}",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)
                for (sx,sy) in studs_in:
                    cv2.circle(dbg,(int(sx),int(sy)),3,(255,0,0),-1)
                roi_id+=1

                cv2.rectangle(vis,(x,y),(x+w,y+h),(0,255,255),2)
                cv2.putText(vis,f"{color} {label}",(x,y-5),
                            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,255),2)

                # --- Draw centroid in black ---
                cx, cy = int(x + w / 2), int(y + h / 2)
                cv2.circle(dbg, (cx, cy), 4, (0, 0, 0), -1)
                cv2.circle(vis, (cx, cy), 4, (0, 0, 0), -1)


                all_rows.append({"Kit":name,"Color":color,"Size":label,"Studs":n_studs})

        # Save and show debug views
        cv2.imwrite(f"results/debug/{name}_rois.png", dbg)
        cv2.imwrite(f"results/annotated/{name}_bricks.png", vis)
        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(dbg, cv2.COLOR_BGR2RGB))
        plt.title(f"{name} — ROI Debug (red boxes = clusters, blue dots = studs)")
        plt.axis("off")
        plt.show()
        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title(f"{name} — Bricks Labeled (yellow boxes)")
        plt.axis("off")
        plt.show()


    # ===== Pretty summary print: kit name & format shown only once per group =====
    if all_rows:
        df = pd.DataFrame(all_rows)

        # Aggregate results
        summary = (
            df.groupby(["Kit", "Size", "Color"])
              .size()
              .reset_index(name="quantity")
              .sort_values(["Kit", "Size", "Color"])
        )

        print("\n📊 Final Results — Bricks per Kit\n")

        pretty_blocks = []
        for kit_name, sub in summary.groupby("Kit", sort=False):
            sub = sub.copy()
            # Rename columns only once
            sub = sub.rename(columns={"Kit": "kit name", "Size": "format", "Color": "color"})

            # Make 'kit name' column blank except for first row
            sub.loc[sub.index[0], "kit name"] = kit_name
            sub.loc[sub.index[1]:, "kit name"] = ""

            # Remove repeated formats within the same kit
            last_fmt = None
            for i in sub.index:
                fmt = sub.at[i, "format"]
                if fmt == last_fmt:
                    sub.at[i, "format"] = ""
                else:
                    last_fmt = fmt

            pretty_blocks.append(sub[["kit name", "format", "color", "quantity"]])

        pretty_summary = pd.concat(pretty_blocks, ignore_index=True)

        # Display neatly without index column
        display(pretty_summary.style.hide(axis="index"))

        # Save full detailed CSV (with all values)
        summary.to_csv("results/kit_summary.csv", index=False)
        print("✅ Summary saved to results/kit_summary.csv")
    else:
        print("No results to summarize.")


def classify_new_kits(folder="./data/Kit",
                      reference_files=None,
                      test_files=None):
    """
    Classifies test kits against known reference kits by comparing brick distributions
    (size + color quantities only).
    """

    if reference_files is None:
        reference_files = ["kit1.png", "kit2.png", "kit3.png"]
    if test_files is None:
        test_files = ["ImageA_kit.png", "ImageB_kit.png", "ImageC_kit.png"]

    print(f"🔍 Running classification in folder: {folder}")
    process_all_kits(folder)

    df_all = pd.read_csv("results/kit_summary.csv")

    # --- Normalize column names (your table uses Kit, Size, Color, quantity) ---
    if "quantity" not in df_all.columns:
        # fallback capitalization check
        for col in df_all.columns:
            if col.lower() == "quantity":
                df_all.rename(columns={col: "quantity"}, inplace=True)

    df_all.rename(columns={
        "Kit": "kit",
        "Size": "size",
        "Color": "color"
    }, inplace=True)

    # --- Split reference and test kits ---
    reference_df = df_all[df_all["kit"].isin(reference_files)]
    test_df = df_all[df_all["kit"].isin(test_files)]

    print(f"\n📦 Found {reference_df['kit'].nunique()} reference kits and "
          f"{test_df['kit'].nunique()} test kits.\n")

    # --- Build kit compositions ---
    def build_comp(df):
        out = {}
        for kit, sub in df.groupby("kit"):
            comp = {f"{r['size']}_{r['color']}": int(r['quantity']) for _, r in sub.iterrows()}
            out[kit] = comp
        return out

    reference = build_comp(reference_df)
    test_comps = build_comp(test_df)

    # --- Compare test kits to references ---
    print("🧠 Comparing test kits to reference kits...\n")
    results = []
    for test_name, test_comp in test_comps.items():
        best_match, best_score = None, 0
        for ref_name, ref_comp in reference.items():
            matches = sum(1 for k, v in test_comp.items() if k in ref_comp and ref_comp[k] == v)
            total = max(len(ref_comp), len(test_comp))
            score = matches / total if total > 0 else 0
            if score > best_score:
                best_match, best_score = ref_name, score

        status = "✅ Exact match" if best_score == 1.0 else (
            "⚠️ Partial match" if best_score > 0 else "❌ No match"
        )
        print(f"{test_name:<20} → {best_match or 'None':<10} ({best_score*100:5.1f}% similarity)  {status}")
        results.append({
            "image": test_name,
            "match": best_match or "None",
            "similarity": round(best_score, 3),
            "status": status
        })

    df_results = pd.DataFrame(results)
    df_results.to_csv("results/kit_comparison.csv", index=False)
    print("\n📊 Classification summary saved → results/kit_comparison.csv")
    display(df_results)
    return df_results


classify_new_kits(
    folder="./data/Kit",
    reference_files=["kit1.png", "kit2.png", "kit3.png"],
    test_files=["ImageA_kit.png", "ImageB_kit.png", "ImageC_kit.png"]
)


## Results - Answer to Question 3C)

Previous code show kit detection and association between test and reference kits.

## Results - Answer to Question 3D)

For a proper image detection it is very important the calibration process and to ensure the conditions from the calibration process maitained along the time.

The image process is very sensitive,  changing of light, distance or angle os view can generate errors in the detection process (because of shadows, glare and other efects).

Also the process can take advantage if the position of the pieces is made more regular (for example all pieces in parallel and with a good degree of separation). It would also benefit the process if the background (the surface where the pieces are) is not of the same color of the pieces - e.g. the white pieces were dificult to detect and only with a combination of maks and studs it was possible to do).

### EXTRA

Use a controlled imaging rig: enclose the belt, add uniform diffuse LED lighting with cross-polarizers, a matte high-contrast background, and global-shutter cameras synced to short strobes to freeze motion and kill glare.
Run routine calibration: geometric (checkerboard) and lens distortion, plus color calibration with a reference chart; lock white balance/exposure per line, monitor temperature drift, and re-calibrate on a schedule.
Mitigate photometric effects by avoiding shadows and specular highlights, using HDR or exposure bracketing for very shiny/dark pieces, and keeping a linear (gamma-aware) color pipeline so thresholds stay stable.
Improve separability before vision: add singulation/spreader rails to reduce overlaps, consider top+side (or 3D) views to see studs and height, and maintain a data-driven re-tuning loop (periodic re-labeling and threshold updates) based on production images.



## 4️⃣ Faulty Kits

In [ ]:
# AUXILIARY FUNCTION TO INSPECT IMAGE CONDITIONS

import cv2, numpy as np, matplotlib.pyplot as plt, seaborn as sns, glob, os

def inspect_image_conditions(folder="./data/Fault", n_show=3):
    """
    Inspects basic image statistics (brightness, contrast, color histograms, stud scale hints)
    to guide parameter tuning.
    """
    files = sorted(glob.glob(os.path.join(folder, "*.png")))
    print(f"📂 Found {len(files)} images in {folder}\n")
    if not files:
        return

    stats = []
    for path in files:
        img = cv2.imread(path)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        mean_v = hsv[...,2].mean()
        mean_s = hsv[...,1].mean()
        sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()

        stats.append({
            "image": os.path.basename(path),
            "mean_V": mean_v,
            "mean_S": mean_s,
            "sharpness": sharpness,
            "height": img.shape[0],
            "width": img.shape[1]
        })

    df = pd.DataFrame(stats)
    display(df.style.background_gradient(cmap="viridis", subset=["mean_V","mean_S","sharpness"]))

    # ---- Brightness & saturation comparison ----
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    sns.barplot(df, x="image", y="mean_V", color="gold")
    plt.title("Mean Brightness (V channel)")
    plt.xticks(rotation=45, ha="right")

    plt.subplot(1,2,2)
    sns.barplot(df, x="image", y="mean_S", color="limegreen")
    plt.title("Mean Saturation (S channel)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    # ---- Example histograms and circle size hint ----
    for i, path in enumerate(files[:n_show]):
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        plt.figure(figsize=(12,5))
        plt.subplot(1,2,1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"{os.path.basename(path)} — Preview")
        plt.axis("off")

        plt.subplot(1,2,2)
        plt.hist(gray.ravel(), bins=50, color='gray')
        plt.title("Grayscale histogram (contrast indicator)")
        plt.xlabel("Intensity"); plt.ylabel("Pixels")
        plt.tight_layout()
        plt.show()

    print("\n💡 Interpretation tips:")
    print(" - Lower mean_V → darker images → increase exposure/brightness correction or lower Hough param1.")
    print(" - Lower sharpness → blurrier images → raise minRadius or preprocess with unsharp mask.")
    print(" - Very low mean_S → faded colors → relax HSV saturation thresholds.")
    print(" - Check grayscale histograms: narrow peaks mean low contrast.")


In [ ]:
# INSPECT IMAGES

inspect_image_conditions("./data/Fault")


In [ ]:
# ============================================================
# 🧱 LEGO Fault Set — per-color brick detection & fault report
# ============================================================
# Fixes: Tuned white color levels for mask.
# Processes ./data/Fault, displays annotated detections,
# outputs kit_summary_fault.csv and kit_comparison_fault.csv
# ============================================================

import os, glob, shutil, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial import cKDTree

# ======= CONFIG =======
PX_PER_MM = 0.330046
VALID_STUD_COUNTS = {2: "2x1", 4: "2x2", 8: "4x2", 12: "6x2"}
STUD_MISS_TOL = 1
EPS_MULTIPLIER = 1.23
ROI_MARGIN = 10
COLORS = ["red", "blue", "green", "yellow", "white"]

# for studs detection
PARAM1 = 70   # (was 90)
PARAM2 = 18   # (was 20)
MINR, MAXR = 6, 18   # (was 5, 16)

# for removing close studs
MIN_DIST = 35             # Tuned from 30 to 35


# ======= UTILS / FUNCTIONS =======
def reset_results():
    base = "results"
    if os.path.exists(base):
        shutil.rmtree(base)
    os.makedirs("results/annotated", exist_ok=True)
    os.makedirs("results/debug", exist_ok=True)

def mm2_from_px_area(px_area):
    return float(px_area) * (PX_PER_MM ** 2)

"""
===============================================================================
🔍 STUD DETECTION CONFIGURATION — FINAL TUNED PARAMETERS
===============================================================================

Function: detect_all_studs(img_bgr, color_hint=None)

Detection method:
- Converts to grayscale → equalizes histogram → median blur
- Applies color-specific Hough Circle parameters
- For white, applies a sharpening filter before Hough (boost edges)

Color | Hough Parameters | Notes
------|------------------|------
🔴 red     | param1=55, param2=12, minR=5,  maxR=18 | Low thresholds to catch weak red edges.
🟡 yellow  | param1=65, param2=17, minR=5,  maxR=16 | Balanced; combined with eps*1.10 in DBSCAN.
🟢 green   | param1=90, param2=18, minR=5,  maxR=16 | Moderate sensitivity.
🔵 blue    | param1=90, param2=18, minR=5,  maxR=16 | Stable; limited noise.
⚪ white   | param1=75, param2=17, minR=5,  maxR=16 | Edge-sharpened preprocessing.

Additional global parameters:
--------------------------------
PX_PER_MM         = 0.330046      # pixel–to–mm conversion factor
VALID_STUD_COUNTS = {2:'2x1', 4:'2x2', 8:'4x2', 12:'6x2'}
STUD_MISS_TOL     = 1             # Snap tolerance (±1 stud)
EPS_MULTIPLIER    = 1.10 (yellow) # Slightly larger DBSCAN eps for yellow
MIN_DIST          = 30            # Minimum inter-stud distance
ROI_MARGIN        = 10            # Padding for bounding boxes

===============================================================================
"""
# function to detect all studs in image with color hint
def detect_all_studs(img_bgr, color_hint=None):
    """
    Detect LEGO studs using tuned HoughCircles, adaptive per color_hint.
    color_hint can be one of: 'red', 'yellow', 'green', 'blue', 'white', or None.
    """

    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    gray = cv2.medianBlur(gray, 3)

    # --- Default parameters ---
    param1 = PARAM1   # upper threshold for edge detection
    param2 = PARAM2   # accumulator threshold (lower → more circles)
    minR, maxR = MINR, MAXR

    # --- Adjust based on color ---
    if color_hint == 'red':
        param1 = 55   # lower edge threshold → catches weak edges
        param2 = 12   # even more sensitive accumulator
        minR, maxR = 5, 18
    elif color_hint == 'yellow':
        param2 = 17   # stricter (avoid false positives) 23 was 18
        param1 = 65   # was 100 com 80 melhorou was 80
    elif color_hint == 'green':
        param2 = 18   # was 20
    elif color_hint == 'blue':
        param2 = 18   # was 20
    elif color_hint == 'white':
        param2 = 20   # was 22

    circles = cv2.HoughCircles(
        gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=18,
        param1=param1, param2=param2,
        minRadius=minR, maxRadius=maxR
    )

    if circles is None:
        return np.empty((0, 2), dtype=np.float32)

    C = np.uint16(np.around(circles[0]))
    return C[:, :2].astype(np.float32)

# function to remove stud that is too close to other
def remove_too_close_studs(studs, min_dist=8):
    if len(studs) < 3: return studs
    keep = []
    used = np.zeros(len(studs), dtype=bool)
    for i in range(len(studs)):
        if used[i]: continue
        keep.append(studs[i])
        for j in range(i+1, len(studs)):
            if np.linalg.norm(studs[i]-studs[j]) < min_dist:
                used[j] = True
    return np.array(keep, dtype=np.float32)

# function to estimate eps from all points
def estimate_eps_from_spacing(points):
    if len(points) < 2: return 40.0
    tree = cKDTree(points)
    dists, _ = tree.query(points, k=2)
    nn = np.median(dists[:, 1])
    return float(max(EPS_MULTIPLIER * nn, 25.0))

# function to create ROI boxes from clusters
def boxes_from_stud_clusters(points, labels, margin=ROI_MARGIN):
    rois = []
    for lab in sorted(set(labels)):
        if lab == -1: continue
        P = points[labels == lab]
        x1, y1 = np.min(P, axis=0)
        x2, y2 = np.max(P, axis=0)
        x1, y1 = int(x1 - margin), int(y1 - margin)
        x2, y2 = int(x2 + margin), int(y2 + margin)
        rois.append((x1, y1, max(1,x2-x1), max(1,y2-y1), lab))
    return rois

# function to remove overlap ROI
def remove_overlaps_keep_largest(rois, iou_thresh=0.4, dist_ratio=0.6):
    """Gentle overlap filtering to preserve small bricks."""
    def inter_over_min(a, b):
        ax1, ay1, aw, ah = a[:4]
        bx1, by1, bw, bh = b[:4]
        ax2, ay2 = ax1 + aw, ay1 + ah
        bx2, by2 = bx1 + bw, by1 + bh
        xA, yA = max(ax1, bx1), max(ay1, by1)
        xB, yB = min(ax2, bx2), min(ay2, by2)
        inter = max(0, xB - xA) * max(0, yB - yA)
        areaA, areaB = aw * ah, bw * bh
        return inter / max(1, min(areaA, areaB))

    keep = []
    # Sort by area descending — keep bigger first
    for r in sorted(rois, key=lambda x: x[2] * x[3], reverse=True):
        if all(inter_over_min(r, k) < iou_thresh for k in keep):
            keep.append(r)
    return keep

# function to adjust stud count (it helps in case of missing stud detection)
def snap_stud_count(n):
    """
    Snap stud count to nearest valid size (2, 4, 8, 12)
    with ±STUD_MISS_TOL tolerance, preferring upward snaps on ties.
    """

    n = int(n)

    if n == 6:
      return 4   # treat 6 as an overcounted 2×2 brick

    candidates = np.array(sorted(VALID_STUD_COUNTS.keys()))  # [2, 4, 8, 12]
    diffs = np.abs(candidates - n)
    min_diff = diffs.min()

    # candidates with same min difference
    nearest_candidates = candidates[diffs == min_diff]
    # prefer larger when tied
    nearest = int(nearest_candidates.max())

    if abs(nearest - n) <= STUD_MISS_TOL:
        return nearest
    return None

"""
===============================================================================
🎨 COLOR MASK CONFIGURATION — FINAL TUNED PARAMETERS
===============================================================================

Each color is extracted in HSV space with specific thresholds and morphology.
These values were tuned empirically for the current dataset of LEGO kit images.

Color | HSV Lower (H,S,V) | HSV Upper (H,S,V) | Morphology | Notes
------|-------------------|-------------------|-------------|------
🔴 red     | (0,100,60) & (170,100,60) | (10,255,255) & (180,255,255) | — | Two ranges handle hue wraparound. Very sensitive (weak edges).
🟡 yellow  | (10,90,90) | (34,255,255) | close(7×7) → dilate(7×7) | Avoids overlap with green. Dilation keeps edge studs visible.
🟢 green   | (40,60,50) | (85,255,255) | close(7×7) | Balanced — minimal false positives.
🔵 blue    | (90,60,40) | (130,255,255) | close(7×7) | Stable under mild shadows.
⚪ white   | (0,0,200) | (180,60,255) | close(5×5) → open(5×5) → erode(3×3) | Tight V/S limits remove reflections & background glare.

Morphology legend:
- close(): fills small holes inside the mask (connects studs)
- open(): removes isolated bright noise pixels
- erode(): shrinks glare halos and prevents brick merging

===============================================================================
"""
# function to apply color mask
def make_color_mask(img_bgr, color):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    if color == "yellow":
        lower, upper = (10, 80, 80), (35, 255, 255)       #(10, 100, 100), (35, 255, 255)

    elif color == "red":
        lower1, upper1 = (0, 100, 60), (10, 255, 255)     #(0, 120, 70), (10, 255, 255)
        lower2, upper2 = (170, 100, 60), (180, 255, 255)  #(170, 120, 70), (180, 255, 255)
        return (cv2.inRange(hsv, np.array(lower1), np.array(upper1)) |
                cv2.inRange(hsv, np.array(lower2), np.array(upper2)))
    elif color == "blue":
        lower, upper = (90, 60, 40), (130, 255, 255)      #(90, 80, 50), (130, 255, 255)
    elif color == "green":
        lower, upper = (35, 50, 40), (85, 255, 255)       #(40, 60, 50), (85, 255, 255)
    elif color == 'white':

        # White has very low saturation and high brightness
        lower = (0,0,200)       # This is what works better for this case - before was (0, 0, 180)
        upper = (180, 60, 255)  # This is what works better for this case - before was (180, 40, 255)
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))

        # Morphological cleaning to remove noise and fill holes
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # Extra step: erode slightly to avoid merging with bright reflections
        mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=1)

        return mask

    else:
        raise ValueError(f"Unknown color: {color}")
    mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))    # was 9,9
    return mask

# function to keep only studs that are inside color mask
def keep_studs_inside_mask(studs, mask):
    h, w = mask.shape
    kept = []
    for (x, y) in studs:
        xi, yi = int(round(x)), int(round(y))
        if 0 <= xi < w and 0 <= yi < h and mask[yi, xi] > 0:
            kept.append([x, y])
    return np.array(kept, dtype=np.float32)

# === Helper: clean results folders ===
def reset_results(base_results):
    import shutil
    for sub in ["debug", "annotated"]:
        path = os.path.join(base_results, sub)
        os.makedirs(path, exist_ok=True)
        for f in os.listdir(path):
            os.remove(os.path.join(path, f))

# === Main processing function ===
def process_all_kits(folder="./data/Kit", results_name="kit"):

    base_results = f"results_{results_name}"
    os.makedirs(base_results, exist_ok=True)
    reset_results(base_results)

    all_rows = []
    for kit_path in sorted(glob.glob(os.path.join(folder, "*.png"))):

        name = os.path.basename(kit_path)
        print(f"\n🔹 Processing {name}")
        img = cv2.imread(kit_path)

        dbg, vis = img.copy(), img.copy()
        roi_id = 1

        for color in COLORS:
            mask = make_color_mask(img, color)

            studs_raw = detect_all_studs(img, color_hint=color)
            # ADD PRINT - debug
            print(f"[{color}] raw studs={len(studs_raw)}")

            studs = keep_studs_inside_mask(studs_raw, mask)
            # ADD PRINT - debug
            print(f"   after mask filter: {len(studs)}")

            studs = remove_too_close_studs(studs, min_dist=MIN_DIST)
            # ADD PRINT - debug
            print(f"   after too-close filter: {len(studs)}")

            if len(studs)==0:
                continue

            # CLUSTER PROCESS

            eps = estimate_eps_from_spacing(studs)

            #FIX FOR YELLOW (tunning)
            if color == "yellow":
              eps *= 1.10  # increase by 25% only for yellow

            labels = DBSCAN(eps=eps, min_samples=2).fit_predict(studs)

            # ADD PRINT - debug
            print(f"[{color}] studs={len(studs)}, unique DBSCAN labels={set(labels)} (eps={eps:.1f})")

            rois = boxes_from_stud_clusters(studs, labels)
            rois = remove_overlaps_keep_largest(rois)

            for (x,y,w,h,lab) in rois:
                studs_in = studs[labels==lab]
                n_studs = len(studs_in)
                snapped = snap_stud_count(n_studs)
                label = VALID_STUD_COUNTS.get(snapped, "unknown")

                # ADD PRINT - debug
                print(f"   cluster {lab}: {n_studs} studs → snapped={snapped}, label={label}")

                cv2.rectangle(dbg,(x,y),(x+w,y+h),(0,0,255),2)
                cv2.putText(dbg,f"ROI {roi_id}",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)
                for (sx,sy) in studs_in:
                    cv2.circle(dbg,(int(sx),int(sy)),3,(255,0,0),-1)
                roi_id+=1

                cv2.rectangle(vis,(x,y),(x+w,y+h),(0,255,255),2)
                cv2.putText(vis,f"{color} {label}",(x,y-5),
                            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,255),2)

                cx, cy = int(x + w / 2), int(y + h / 2)
                cv2.circle(dbg, (cx, cy), 4, (0, 0, 0), -1)
                cv2.circle(vis, (cx, cy), 4, (0, 0, 0), -1)

                all_rows.append({"Kit":name,"Color":color,"Size":label,"Studs":n_studs})

        # Save images
        cv2.imwrite(f"{base_results}/debug/{name}_rois.png", dbg)
        cv2.imwrite(f"{base_results}/annotated/{name}_bricks.png", vis)

        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(dbg, cv2.COLOR_BGR2RGB))
        plt.title(f"{name} — ROI Debug (red boxes = clusters, blue dots = studs)")
        plt.axis("off")
        plt.show()

        plt.figure(figsize=(10,8))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title(f"{name} — Bricks Labeled (yellow boxes)")
        plt.axis("off")
        plt.show()

    if all_rows:
        df = pd.DataFrame(all_rows)
        summary = (
            df.groupby(["Kit", "Size", "Color"])
              .size()
              .reset_index(name="quantity")
              .sort_values(["Kit", "Size", "Color"])
        )

        print("\n📊 Final Results — Bricks per Kit\n")

        pretty_blocks = []
        for kit_name, sub in summary.groupby("Kit", sort=False):
            sub = sub.copy()
            sub = sub.rename(columns={"Kit": "kit name", "Size": "format", "Color": "color"})
            sub.loc[sub.index[0], "kit name"] = kit_name
            sub.loc[sub.index[1]:, "kit name"] = ""
            last_fmt = None
            for i in sub.index:
                fmt = sub.at[i, "format"]
                if fmt == last_fmt:
                    sub.at[i, "format"] = ""
                else:
                    last_fmt = fmt
            pretty_blocks.append(sub[["kit name", "format", "color", "quantity"]])

        pretty_summary = pd.concat(pretty_blocks, ignore_index=True)
        display(pretty_summary.style.hide(axis="index"))

        csv_path = f"{base_results}/kit_summary_{results_name}.csv"
        summary.to_csv(csv_path, index=False)
        print(f"✅ Summary saved to {csv_path}")

    else:
        print("No results to summarize.")


def classify_new_kits(folder="./data/Fault",
                      results_name="fault",
                      reference_files=None,
                      test_files=None):

    base_results = f"results_{results_name}"
    os.makedirs(base_results, exist_ok=True)

    print(f"🔍 Running classification in folder: {folder}")
    process_all_kits(folder, results_name=results_name)

    df_all = pd.read_csv(f"{base_results}/kit_summary_{results_name}.csv")

    df_all.rename(columns={"Kit": "kit", "Size": "size", "Color": "color"}, inplace=True)

    reference_df = df_all[df_all["kit"].isin(reference_files)]
    test_df = df_all[df_all["kit"].isin(test_files)]

    print(f"\n📦 Found {reference_df['kit'].nunique()} reference kits and "
          f"{test_df['kit'].nunique()} test kits.\n")

    def build_comp(df):
        out = {}
        for kit, sub in df.groupby("kit"):
            comp = (
                sub.groupby(["size", "color"])["quantity"]
                  .sum()
                  .apply(int)
                  .to_dict()
            )
            # Flatten to "size_color" keys
            comp = {f"{size}_{color}": qty for (size, color), qty in comp.items()}
            out[kit] = comp
        return out

    reference = build_comp(reference_df)
    test_comps = build_comp(test_df)


    print("🧠 Comparing test kits to reference kits...\n")
    results = []

    for test_name, test_comp in test_comps.items():
        best_match, best_score, best_diff_comment = None, 0, ""

        for ref_name, ref_comp in reference.items():
            # --- Compare test vs reference ---
            matches = sum(1 for k, v in test_comp.items() if k in ref_comp and ref_comp[k] == v)
            total = max(len(ref_comp), len(test_comp))
            score = matches / total if total > 0 else 0

            # --- Keep best match ---
            if score > best_score:
                best_match, best_score = ref_name, score

                # --- Compute differences ---
                missing, exceeding = [], []

                # Bricks missing or lower count than reference
                for k, v in ref_comp.items():
                    if k not in test_comp:
                        missing.append(f"{k} (x{v} missing)")
                    elif test_comp[k] < v:
                        diff = v - test_comp[k]
                        missing.append(f"{k} (x{diff} missing)")

                # Bricks exceeding or absent in reference
                for k, v in test_comp.items():
                    if k not in ref_comp:
                        exceeding.append(f"{k} (x{v} extra)")
                    elif test_comp[k] > ref_comp[k]:
                        diff = test_comp[k] - ref_comp[k]
                        exceeding.append(f"{k} (x{diff} extra)")

                # Build human-readable comment
                comments = []
                if missing:
                    comments.append("Missing: " + ", ".join(missing))
                if exceeding:
                    comments.append("Extra: " + ", ".join(exceeding))
                best_diff_comment = " | ".join(comments) if comments else "All bricks match."

        # --- Assign status and print summary ---
        if best_score == 1.0:
            status = "✅ Exact Match"
        elif best_score > 0:
            status = "❌ Faulty Kit"   # renamed from Partial Match
        else:
            status = "❌ No Match"

        print(f"{test_name:<20} → {best_match or 'None':<10} ({best_score*100:5.1f}% similarity)  {status}")
        print(f"   ↪ {best_diff_comment}")

        results.append({
            "image": test_name,
            "match": best_match or "None",
            "similarity": round(best_score, 3),
            "status": status,
            "comment": best_diff_comment
        })

    # --- Save final DataFrame ---
    df_results = pd.DataFrame(results)
    out_csv = f"{base_results}/kit_comparison_{results_name}.csv"
    df_results.to_csv(out_csv, index=False)

    print(f"\n📊 Classification summary saved → {out_csv}")
    display(df_results)
    return df_results


#RUN PROCESS

final_result = classify_new_kits(
                  folder="./data/Fault",
                  results_name="fault",
                  reference_files=["kitA.png", "kitB.png", "kitC.png"],
                  test_files=["ImageA_fault.png", "ImageB_fault.png", "ImageC_fault.png",
                              "imageD_fault.png", "ImageE_fault.png", "ImageF_fault.png"]
                  )

## Results - Answer to Question 4)

Previous code show kit detection and association between test and reference kits (for FAULT images folder).

In [ ]:
# AUXILIARY FUNCTION TO OBSERVE WHAT IS BEING DONE BY MASK AND STUD DETECTION

def inspect_detection_steps(img_path, color="yellow"):
    """
    Visualize mask quality and stud detection overlay for a single color.
    Helps decide if issues come from HSV mask or Hough parameters.
    """
    img = cv2.imread(img_path)
    #hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    mask = make_color_mask(img, color)
    studs = detect_all_studs(img, color_hint=color)
    studs = keep_studs_inside_mask(studs, mask)
    studs = remove_too_close_studs(studs, min_dist=MIN_DIST)

    # Prepare overlay visualization
    overlay = img.copy()
    overlay[mask > 0] = (0.3 * overlay[mask > 0] + 0.7 * np.array([0,255,255])).astype(np.uint8)
    for (x, y) in studs:
        cv2.circle(overlay, (int(x), int(y)), 5, (255, 0, 0), -1)

    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Original Image"); plt.axis("off")

    plt.subplot(1,3,2)
    plt.imshow(mask, cmap="gray")
    plt.title(f"Mask for {color}"); plt.axis("off")

    plt.subplot(1,3,3)
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title(f"Detected studs on {color} mask overlay")
    plt.axis("off")
    plt.show()

    print(f"🔹 Studs detected: {len(studs)}")

In [ ]:
# INSPECTION TO DETECT EDGE CASES

inspect_detection_steps("./data/Fault/ImageA_fault.png", color="white")

inspect_detection_steps("./data/Fault/ImageA_fault.png", color="yellow")

inspect_detection_steps("./data/Fault/ImageA_fault.png", color="red")

inspect_detection_steps("./data/Fault/ImageE_fault.png", color="yellow")

inspect_detection_steps("./data/Fault/ImageB_fault.png", color="yellow")
